In [2]:
def load_patient_data_frame(samples_excluded_H,samples_excluded_MDS,opt_mutations_of_interest,opt_sample_ID,path):
    
    cd = os.getcwd()
    os.chdir(path)
    os.chdir('../')
    #load patient data:
    df_PD = pd.read_csv('./data_analysis/2020_11_23_Patiententabelle.txt', sep="\t",na_values=['NaN'], encoding = "ISO-8859-1")
    #load FACS data:
    df_FACS = pd.read_csv('./data_analysis/hematopoiesis_FACS_2020_08_data.txt', sep=" ", na_values=['NaN'])
    os.chdir(cd)
    
    
    df_PD.rename(columns={'sample_number':'Sample_ID'}, inplace=True)
    #remove certain samples which should be excluded from plots/ analysis
    for s in samples_excluded_H:
        df_PD = df_PD[df_PD['Sample_ID']!="H"+s]
    for s in samples_excluded_MDS:
        df_PD = df_PD[df_PD['Sample_ID']!="MDS"+s]
    #reset index:
    df_PD.reset_index(inplace=True,drop=True)
    # add column VAF for each mutation
    if opt_mutations_of_interest == 'bulk':
        mut_ending = '_B'        
    else:
        mut_ending = '_S'
    mutation_str = df_PD.columns[df_PD.columns.str.endswith(mut_ending)]
    if opt_mutations_of_interest == 'combined':
        m_str = [mut_str[:-2] for mut_str in mutation_str]
        mutation_str = m_str
    N_mut=list()
    for mut_str in mutation_str:
        df_PD[mut_str+'_VAF'] = df_PD[mut_str].copy()
        df_PD[mut_str+'_VAF'] = np.select([df_PD[mut_str+'_VAF'].values=='no'],['0'],default = df_PD[mut_str])    
    #print(df[['Sample_ID',mut_str+'_VAF']])
    M = df_PD[df_PD.columns[df_PD.columns.str.endswith('_VAF')]].values
    N_mut.append(np.nansum(np.transpose(M.astype(float)>0),axis=0))
    #N_mut=list()
    #for mut_str in mutation_str:
    #    df_PD[mut_str+'_VAF'] = df_PD[mut_str]
    #    df_PD[mut_str+'_VAF'] = np.select([df_PD['Sample_ID'].str.startswith('H'),df_PD[mut_str+'_VAF'].values=='no'],['0','0'],default = df_PD[mut_str])        
    #M = df_PD[df_PD.columns[df_PD.columns.str.endswith('_VAF')]].values
    #N_mut.append(np.nansum(np.transpose(M.astype(float)>0),axis=0))
    
    # add column mutation counts
    df_PD['Mutation_counts'] = np.transpose(N_mut)

    
    #remove measuraments later than day 7
    df_FACS = df_FACS[df_FACS['Tag'] <=7]
    #change columns status and sample name
    df_FACS['Status'] = df_FACS['Gruppe_2_categories'].map({'healthy': 'H', 'MDS': 'MDS'})
    df_FACS['sample_name']=df_FACS['sample_name'].str.replace(' ', '')
    #remove certain samples which should be excluded from plots/ analysis
    if samples_excluded_H:
        for s in samples_excluded_H:
            df_FACS = df_FACS[(df_FACS['sample_name']!=s)]
    if samples_excluded_MDS:
        for s in samples_excluded_MDS:
            df_FACS = df_FACS[(df_FACS['sample_name']!=s)]
    #reset index:
    df_FACS.reset_index(inplace=True,drop=True)
    #change sample name for repetitions
    df_FACS['Sample_ID']=df_FACS['Status'].copy()+df_FACS['sample_name'].copy().str[0:3]
    #df_FACS['sample_name'] = np.select([df_FACS['repetition']==1,df_FACS['repetition']==2],[df_FACS['sample_name']+"_1",df_FACS['sample_name']+"_2"])
    #add column number of datapoints
    idx = df_FACS['sample_name'].str.find('_')
    sample_numbers = [sample_nr[:idx[i]] for i,sample_nr in enumerate(df_FACS['sample_name'])]
    if opt_sample_ID=='long':
        df_FACS['Sample_ID_long']=[sample_nr[:idx[i]+2] for i,sample_nr in enumerate(df_FACS['sample_name'])]
        df_FACS['number_data_points'] = [len(df_FACS[df_FACS['Sample_ID_long']==sample_nr[:idx[i]+2]]) for i,sample_nr in enumerate(df_FACS['sample_name'])]
        DF_PD = pd.merge(df_FACS[['Sample_ID','Sample_ID_long','number_data_points']],df_PD, on='Sample_ID', how='inner')
        DF_PD['Sample_ID_long'][DF_PD['Sample_ID'].str.startswith('H')]='H'+DF_PD['Sample_ID_long']
        DF_PD['Sample_ID_long'][DF_PD['Sample_ID'].str.startswith('MDS')]='MDS'+DF_PD['Sample_ID_long']
        DF_PD.drop_duplicates(subset ='Sample_ID_long', keep = 'first', inplace = True) 
        df_PD.loc[df_PD.Sample_ID_long=='H353', 'Sample_ID_simplyfied'] = 'H1*'
    else:
        df_PD.loc[df_PD.Sample_ID=='H353', 'Sample_ID_simplyfied'] = 'H1*'
        df_PD['number_data_points'] = [len(df_FACS[df_FACS['Sample_ID']==s_id]) for s_id in df_PD['Sample_ID']]
        DF_PD = df_PD
    
    df_PD['weight_data'] = df_PD['number_data_points']/sum(df_PD['number_data_points'])
    return DF_PD, mutation_str

In [1]:
def get_par_estimates(path_matlab_result, last_SSF_str, M_names_opt, states, par_transformation,bool_identifiablePars_only,bool_weights,bool_test_par,bool_CIs,bool_boundaries,bool_fit_repetitions_seperately):
    #N_individuals = len(SSF_str)
    if bool_fit_repetitions_seperately == True:
        add_str = '_rep_sep'
    else:
        add_str = ''
    if len(last_SSF_str)==2:
        n=len(last_SSF_str)
    else:
        n=1
    for lssf_id in range(0,n):
        os.chdir(path_matlab_result + last_SSF_str[lssf_id])
        #print(path_matlab_result + last_SSF_str[lssf_id])
        if bool_test_par:
            P = sio.loadmat('ws_parameters_healthy'+add_str+'.mat')
        else:
            if lssf_id==0:
                P = sio.loadmat('ws_parameters_healthy'+add_str+'.mat')
            else:
                P = sio.loadmat('ws_parameters_MDS'+add_str+'.mat')

        transformation_str = P['transformation_str']
        t_id_ratio=np.where(transformation_str[0,:]=='ratio')[0][0]
        
        p_opt = P['PAR_OPT_T']
        
        rate_names_opt = P['rate_names_opt']
        #print(rate_names_opt)
        
        #print(np.char.startswith(rate_names_opt.astype(str),'x0_',start=0,end=None))
        if bool_test_par:
            p_test = P['PAR_TEST_T']
            rate_names_test = P['rate_names_test']
        if par_transformation == 'ratio':
            if len(np.shape(p_opt))==2:
                p_opt[t_id_ratio,:] = p_opt[t_id_ratio,:]/24
                if bool_test_par:
                    p_test[t_id_ratio,:] = p_test[t_id_ratio,:]/24
            elif len(np.shape(p_opt))==3:
                p_opt[t_id_ratio,:,:] = p_opt[t_id_ratio,:,:]/24
                if bool_test_par:
                    p_test[t_id_ratio,:,:] = p_test[t_id_ratio,:,:]/24
        
        if bool_CIs or bool_boundaries or bool_identifiablePars_only or (bool_weights and par_transformation=='lin'):
            CI_l = P['CI_lower']
            CI_u = P['CI_upper']
            p_min = P['par_min']
            p_max = P['par_max']
            if par_transformation == 'ratio':
                if len(np.shape(p_opt))==2:
                    CI_l[t_id_ratio,:] = CI_l[t_id_ratio,:]/24
                    CI_u[t_id_ratio,:] = CI_u[t_id_ratio,:]/24
                    p_min[t_id_ratio,:] = p_min[t_id_ratio,:]/24
                    p_max[t_id_ratio,:] = p_max[t_id_ratio,:]/24
                elif len(np.shape(p_opt))==3:
                    CI_l[t_id_ratio,:,:] = CI_l[t_id_ratio,:,:]/24
                    CI_u[t_id_ratio,:,:] = CI_u[t_id_ratio,:,:]/24
                    p_min[t_id_ratio,:,:] = p_min[t_id_ratio,:,:]/24
                    p_max[t_id_ratio,:,:] = p_max[t_id_ratio,:,:]/24
        
        I_str = P['individuals_str']
        if bool_test_par:
            if last_SSF_str[lssf_id].endswith('weak_noise'):
                Indis = 'weak'
            elif last_SSF_str[lssf_id].endswith('middle_noise'):
                Indis = 'middle'
            elif last_SSF_str[lssf_id].endswith('strong_noise'):
                Indis = 'strong'
            elif last_SSF_str[lssf_id].endswith('realistic_noise'):
                Indis = 'realistic'
            #Indis=last_SSF_str[lssf_id]
            if len(np.shape(p_opt))==2:
                N_individuals=1
            elif len(np.shape(p_opt))==3:
                N_individuals=np.shape(p_opt)[2]
        else:
            if lssf_id==0:
                Indis = list()
                for row in I_str.flatten():
                    Indis.append('H'+row.astype(str)[0])
                N_individuals=len(Indis)
            else:
                Indis = list()
                for row in I_str.flatten():
                    Indis.append('MDS'+row.astype(str)[0])
                N_individuals=len(Indis)
                
        T_ID=np.where(transformation_str[0,:]==par_transformation)[0][0]
        
        if len(np.shape(p_opt))==2:
            [a,b]=np.shape(p_opt)
        elif len(np.shape(p_opt))==3:
            [a,b,c]=np.shape(p_opt)
        #DF_temp = pd.DataFrame(data=['model_'+str(M_names_opt[0])]*(N_individuals),columns=['Model'])
        DF_temp = pd.DataFrame(data=[M_names_opt]*(N_individuals),columns=['Model'])
        DF_temp['Sample_ID_long']=Indis
        DF_temp['Sample_ID']=Indis
        DF_temp['Sample_ID_simplyfied']=Indis
        DF_temp.loc[DF_temp.Sample_ID=='H353', 'Sample_ID_simplyfied'] = 'H1*'
        #DF_temp['Sample_ID']=[name[:-2] for name in Indis] 

        for p_id in range(0,b):
            colName=str(rate_names_opt[0,p_id].item(0).replace('{','').replace('}',''))
            if len(np.shape(p_opt))==2:
                DF_temp[colName] = p_opt[T_ID,p_id]
                if bool_identifiablePars_only and ~(CI_l[T_ID,p_id] > p_min[T_ID,p_id] and CI_u[T_ID,p_id] < p_max[T_ID,p_id]):
                    DF_temp[colName] = float("nan")
                if bool_weights and par_transformation=='lin': 
                    W_CI = np.transpose((p_max[T_ID,p_id]-p_min[T_ID,p_id])/(CI_u[T_ID,p_id]-CI_l[T_ID,p_id]))
                if bool_CIs:
                    DF_temp[colName+'_CI_l'] = CI_l[T_ID,p_id]
                    DF_temp[colName+'_CI_u'] = CI_u[T_ID,p_id]
                if bool_boundaries:
                    DF_temp[colName+'_min'] = p_min[T_ID,p_id]
                    DF_temp[colName+'_max'] = p_max[T_ID,p_id]
                if bool_test_par:
                    DF_temp[colName+'_t'] = p_test[T_ID,p_id]
            elif len(np.shape(p_opt))==3:
                DF_temp[colName] = np.transpose(p_opt[T_ID,p_id,:])
                if bool_identifiablePars_only:
                    idx = [i for i in range(0,c) if ~(CI_l[T_ID,p_id,i] > p_min[T_ID,p_id,i] and CI_u[T_ID,p_id,i] < p_max[T_ID,p_id,i])]
                    DF_temp[colName].iloc[idx] = float("nan")
                if bool_weights and par_transformation=='lin': 
                    W_CI = np.transpose((p_max[T_ID,p_id,:]-p_min[T_ID,p_id,:])/(CI_u[T_ID,p_id,:]-CI_l[T_ID,p_id,:]))
                if bool_CIs:
                    DF_temp[colName+'_CI_l'] = np.transpose(CI_l[T_ID,p_id,:])    
                    DF_temp[colName+'_CI_u'] = np.transpose(CI_u[T_ID,p_id,:]) 
                if bool_boundaries:
                    DF_temp[colName+'_min'] = np.transpose(p_min[T_ID,p_id,:])
                    DF_temp[colName+'_max'] = np.transpose(p_max[T_ID,p_id,:])
                if bool_test_par:
                    DF_temp[colName+'_t'] = np.transpose(p_test[T_ID,p_id,:])
            if bool_weights and par_transformation=='lin': 
                colName_W_CI=str(rate_names_opt[0,p_id].item(0).replace('{','').replace('}',''))+'_w_CI'
                DF_temp[colName_W_CI] = W_CI/np.sum(W_CI)
        if lssf_id>0:
            DF_PAR = pd.concat([DF_PAR,DF_temp], axis=0, ignore_index=True)
        else:
            DF_PAR = DF_temp 
    for rate in DF_PAR.columns:
        #DF_PAR = DF_PAR.replace('NaN',float("nan"), regex=True)
        if rate != 'Model' and rate != 'Sample_ID' and rate != 'Sample_ID_long' and rate != 'Sample_ID_simplyfied':
            DF_PAR[rate]=pd.to_numeric(DF_PAR[rate])
    ##drop samples that were excluded
    #DF_PAR.drop([sample.index for sample in df_PAR['Sample_ID_long'] if 'H'+sample in samples_excluded_H], inplace=True)
    #DF_PAR.drop([sample.index for sample in df_PAR['Sample_ID_long'] if 'MDS'+sample in samples_excluded_MDS], inplace=True)
    return DF_PAR;

In [4]:
def get_dataframe_selection(DF,opt_comparison,opt_sample_ID):
    if opt_sample_ID=='short':
        I_col = 'Sample_ID'
    else:
        I_col = 'Sample_ID_'+opt_sample_ID
    if opt_comparison=='H_age_matched_vs_MDS':
        lab_str1 = 'healthy'
        lab_str2 = 'MDS'
        DF_2=DF[DF[I_col].str.startswith('MDS')].copy()
        min_age_MDS = min(DF_2['age'])
        DF_1=DF[(DF[I_col].str.startswith('H')) & (DF['age']>=min_age_MDS)].copy()
    elif opt_comparison=='H_young_vs_H_aged' or opt_comparison=='H_young_vs_H_aged_with_CHIP':
        lab_str1 = 'young (<=55)'
        lab_str2 = 'aged (>55)'
        DF_1=DF[(DF[I_col].str.startswith('H')) & (DF['age']<=55)].copy()
        DF_2=DF[(DF[I_col].str.startswith('H')) & (DF['age']>55)].copy()
    elif opt_comparison=='H_young_vs_H_aged_without_CHIP':
        lab_str1 = 'young (<=55)'
        lab_str2 = 'aged (>55)'
        mutation_str = DF.columns[DF.columns.str.endswith('VAF')]
        #print(mutation_str)
        DF_1=DF[(DF[I_col].str.startswith('H')) & (np.any(DF[mutation_str].values.astype(np.float)>0,axis=1)==False) & (DF['age']<=55)].copy()
        DF_2=DF[(DF[I_col].str.startswith('H')) & (np.any(DF[mutation_str].values.astype(np.float)>0,axis=1)==False) & (DF['age']>55)].copy()
    elif opt_comparison=='hipReplacement_vs_donor':
        lab_str1 = 'BM donor'
        lab_str2 = 'BM hip replacement'
        DF_1 = DF[DF['sample_type']=='BM donor'].copy()
        DF_2 = DF[DF['sample_type']=='BM hip replacement'].copy()
    elif opt_comparison=='H_age_matched_vs_CHIP_vs_MDS':
        lab_str1 = 'healthy'
        lab_str2 = 'CHIP'
        lab_str3 = 'MDS'
        DF_3=DF[DF[I_col].str.startswith('MDS')].copy()
        min_age_MDS = min(DF_3['age'])
        mutation_str = DF.columns[DF.columns.str.endswith('VAF')]
        DF_1=DF[(DF[I_col].str.startswith('H')) & (np.sum(DF[mutation_str].values.astype(np.float)>0,axis=1)==0) & (DF['age']>=min_age_MDS)].copy()
        DF_2=DF[(DF[I_col].str.startswith('H')) & (np.sum(DF[mutation_str].values.astype(np.float)>0,axis=1)!=0) & (DF['age']>=min_age_MDS)].copy()
    else:
        #DF_1=DF.copy()
        #DF_2=DF.copy()
        print('This option is not implemented, please implement it first in get_dataframe_selection()')
    if opt_comparison=='H_age_matched_vs_CHIP_vs_MDS':
        DF = pd.concat([DF_1,DF_2,DF_3], axis=0, sort=False)
        DF_3.reset_index(inplace=True,drop=True)
    else:
        DF = pd.concat([DF_1,DF_2], axis=0, sort=False)
    DF.reset_index(inplace=True,drop=True)
    DF_1.reset_index(inplace=True,drop=True)
    DF_2.reset_index(inplace=True,drop=True)
    
    if opt_comparison=='H_age_matched_vs_CHIP_vs_MDS':
        return DF, DF_1, DF_2, DF_3, lab_str1, lab_str2, lab_str3
    else:
        return DF, DF_1, DF_2, lab_str1, lab_str2

In [ ]:
def addMetrics2dataframe(DF,par_T,CT):
    if par_T=='lin':
        for ct in CT:
            diff_names_in = DF.columns[np.logical_and(np.logical_and(DF.columns.str.startswith('a_'),DF.columns.str.endswith(ct)),DF.columns != 'a_MLP')]
            diff_names_out = DF.columns[DF.columns.str.startswith('a_'+ct)]
            if ct!='MLP':
                DF[ct +'_log_effective_proliferation_rate'] = np.log(DF['b_'+ct]/DF['g_'+ct])
                DF[ct +'_net_proliferation'] = DF['b_'+ct]-DF['g_'+ct]
                DF[ct+'_cellular_exit_time'] = 1/(np.sum(DF[diff_names_out],axis=1)+DF['g_'+ct])
                DF[ct+'_accumulation_time'] = 1/(np.sum(DF[diff_names_in],axis=1)+DF['b_'+ct]-np.sum(DF[diff_names_out],axis=1)-DF['g_'+ct])
                
                #new:
                DF[ct+'_residence_time'] = 1/(np.sum(DF[diff_names_out],axis=1)-DF['b_'+ct]+DF['g_'+ct]) #comparable to Busch et al 2005
                #T_enter = 1/(np.sum(DF[diff_names_in],axis=1)+DF['b_'+ct])
                #T_exit = 1/(np.sum(DF[diff_names_out],axis=1)+DF['g_'+ct])
                #DF[ct+'_accumulation_time'] = T_exit - T_enter 
                
            #else:
            #    DF[ct +'_net_proliferation'] = DF['b_'+ct]
            #    DF[ct+'_cellular_exit_time'] = 1/(np.sum(DF[diff_names_out],axis=1)+DF['g_'+ct])
    else:
        print('set par_T to lin to enable metric calculation!')
    return DF

In [ ]:
def plot_par_with_CI_vs_individuals(path, main_folder_matlab, F_str, SF_str, SSF_str, last_SSF_str, samples_excluded_H, samples_excluded_MDS, M_names_sim, M_names_opt, states, par_transformation, colors, opt_comparison, rot_deg, opt_sample_ID, bool_fit_repetitions_seperately,bool_test_par,alpha_errorband,opt_plot_number_deregulated_rates,plt,opt_save):
    
    if opt_sample_ID=='short':
        I_col = 'Sample_ID'
    elif opt_sample_ID=='long':
        I_col = 'Sample_ID_long'
    elif opt_sample_ID=='simplyfied':
        I_col = 'Sample_ID_simplyfied'
    L=[]
    bool_weightedMean_Pars=False
    bool_mean_idPars=False
    W=0.2
    #specify dataframe settings
    add_str=''
    bool_weights = False
    if bool_mean_idPars:
        add_str='_mean_identifiable_pars'
    if bool_weightedMean_Pars:
        bool_weights = True
        add_str='_weighted_mean_pars'

    #initialize bools for plotting
    markerSize=15
    bool_plot_legend=False

    #define thresholds
    TH_groups = 2#5 # min of identifiable parameters within group to draw plot
    TH_total = 2#7 # min of identifiable parameters to draw plot
    
    if bool_test_par:
        alpha_val=0.2
    else:
        alpha_val=0.5
    #define positions for text, number of columns and rows
    if (bool_test_par and M_names_opt[0]=='A' and M_names_sim[0]=='A') or (bool_test_par ==False and M_names_opt[0]=='A'):
        n_cols = 3
        x_d = 0.26
        x_p = 0.59
        x_a = 0.79
    elif M_names_opt[0]=='u':
        n_cols = 5
        x_d = 0.34
        x_p = 0.69
        x_a = 0.84 
    else:
        n_cols = 6
        x_d = 0.34
        x_p = 0.69
        x_a = 0.84    
    n_rows = len(states)
    #for f_id in range(0,len(F_str)):
    f_id=0
    for sf_id in range(0,len(M_names_opt)):
        ## get dataframe with results
        #parameters
        path_MATLAB_result = main_folder_matlab + F_str[f_id] + SF_str[sf_id] 

        ## specify figure settings
        if bool_test_par:
            f, axes = plt.subplots(n_rows, n_cols, figsize=(25, 45))
            f.subplots_adjust(hspace = .75, wspace = 0.45)  

            df_par = get_par_estimates(path_MATLAB_result, last_SSF_str, M_names_opt, CT, par_transformation, False,bool_weights,bool_test_par,True,True,bool_fit_repetitions_seperately)

            DF = df_par.copy()
            DF_1=df_par.copy()
            rate_names = df_par.columns[np.logical_and(np.logical_and(np.logical_and(np.logical_and(np.logical_and(np.logical_and(np.logical_and(np.logical_and(np.logical_and(np.logical_and(df_par.columns.str.endswith('_CI_l')==False,df_par.columns.str.endswith('_CI_u')==False),df_par.columns.str.endswith('_min')==False),df_par.columns.str.endswith('_max')==False),df_par.columns.str.endswith('_t')==False),df_par.columns.str.endswith('_w')==False),df_par.columns.str.startswith('_S')==False),df_par.columns.str.startswith('_M')==False),df_par.columns.str.startswith('Model')==False),df_par.columns.str.startswith('x0_')==False),df_par.columns.str.startswith('Sample_ID')==False)] 
            offset = [0,len(DF[I_col])]
            N_indi = [len(DF[I_col]),0]
            I_list = [np.array([i for i in range(0,N_indi[0])]),np.array([i for i in range(0,N_indi[1])])]
            group_str = ['test parameter', '']
        else:
            f, axes = plt.subplots(n_rows, n_cols, figsize=(60, 65))
            f.subplots_adjust(hspace = .45, wspace = 0.35)

            df_par = get_par_estimates(path_MATLAB_result, last_SSF_str, M_names_opt, CT, par_transformation, False ,bool_weights,bool_test_par,True,True,bool_fit_repetitions_seperately)

            #load patientData and define groups of individuals
            df_PD, _ = load_patient_data_frame(samples_excluded_H,samples_excluded_MDS,'combined',opt_sample_ID,path)

            #update weights (include info about number of data points)
            df_PD['weight_data'] = df_PD['number_data_points']/sum(df_PD['number_data_points'])

            #merge data frames
            DF = pd.merge(df_par,df_PD, on=I_col, how='inner')
            #print(df_par.columns)
            rate_names = df_par.columns[np.logical_and(np.logical_and(np.logical_and(np.logical_and(np.logical_and(np.logical_and(np.logical_and(np.logical_and(np.logical_and(np.logical_and(df_par.columns.str.endswith('_CI_l')==False,df_par.columns.str.endswith('_CI_u')==False),df_par.columns.str.endswith('_min')==False),df_par.columns.str.endswith('_max')==False),df_par.columns.str.endswith('_t')==False),df_par.columns.str.endswith('_w')==False),df_par.columns.str.startswith('_S')==False),df_par.columns.str.startswith('_M')==False),df_par.columns.str.startswith('Model')==False),df_par.columns.str.startswith('x0_')==False),df_par.columns.str.startswith('Sample_ID')==False)]  
            #print(rate_names)
            #get dataframes sorted by group comparisons 
            
            if opt_comparison=='H_age_matched_vs_CHIP_vs_MDS':
                DF, DF_1, DF_2, DF_3, lab_str1, lab_str2, lab_str3 = get_dataframe_selection(DF,opt_comparison,opt_sample_ID)
                offset = [0,len(DF_1[I_col]),len(DF_1[I_col])+len(DF_2[I_col])]
                N_indi = [len(DF_1[I_col]),len(DF_2[I_col]),len(DF_3[I_col])]
                I_list = [np.array([i for i in range(0,N_indi[0])]),np.array([i for i in range(0,N_indi[1])]),np.array([i for i in range(0,N_indi[2])])]
                group_str = [lab_str1, lab_str2, lab_str3]
            else:
                DF, DF_1, DF_2, lab_str1, lab_str2 = get_dataframe_selection(DF,opt_comparison,opt_sample_ID)
                offset = [0,len(DF_1[I_col])]
                N_indi = [len(DF_1[I_col]),len(DF_2[I_col])]
                I_list = [np.array([i for i in range(0,N_indi[0])]),np.array([i for i in range(0,N_indi[1])])]
                group_str = [lab_str1, lab_str2]

        f.patch.set_facecolor('white')
        if opt_comparison == 'H_young_vs_H_aged':
            g_end=2
        elif opt_comparison=='H_age_matched_vs_CHIP_vs_MDS':
            g_end=3
        else:
            g_end=1
            w_baseline=np.sum(N_indi)
        s_id = 0 #states
        while s_id < len(CT):
            cols_del = list()
            col_counter = 0
            for r_id,rate in enumerate(rate_names):
                a_m = fnm.fnmatch(rate,'a_'+ CT[s_id] +'*')
                b_m = fnm.fnmatch(rate,'b_'+ CT[s_id] +'*')
                g_m = fnm.fnmatch(rate,'g_'+ CT[s_id] +'*')
                if (a_m or b_m or g_m):
                    if b_m=='b_mat':
                        bool_plot_legend=True
                    if a_m:
                        col = col_counter
                        col_counter = col_counter+1
                    if b_m:
                        col = n_cols-2
                    if g_m:
                        col = n_cols-1
                    #plot weighted group average
                    if bool_weightedMean_Pars and par_transformation == 'lin' and np.sum(N_indi)>TH_total:
                        for g_id in range(0,g_end):
                            if opt_comparison == 'H_young_vs_H_aged' or opt_comparison=='H_age_matched_vs_CHIP_vs_MDS':
                                w_baseline=N_indi[g_id]
                            if g_id==0:
                                ar=DF_1[rate]
                                w_D = DF_1['weight_data']
                                w_CI = DF_1[rate+'_w']
                            elif g_id==1:
                                ar=DF_2[rate]
                                w_D = DF_2['weight_data']
                                w_CI = DF_2[rate+'_w']
                            else:
                                ar=DF_3[rate]
                                w_D = DF_3['weight_data']
                                w_CI = DF_3[rate+'_w']
                            w = (w_D+w_CI)/2
                            weighted_stats = DescrStatsW(ar, weights=w, ddof=0)
                            par_mean_w = weighted_stats.mean
                            par_std_w = weighted_stats.std
                            axes[s_id,col].add_patch(patches.Rectangle((offset[g_id]-W*2, max(DF_1[rate+'_min'][0],par_mean_w-par_std_w*2)), width=w_baseline-1+W*4, height=min((DF_1[rate+'_max'][0]-DF_1[rate+'_min'][0]),4*par_std_w),color=colors[g_id],alpha=0.15,label='weighted mean rate +/- 2 weighted std '+ group_str[g_id]))          
                            axes[s_id,col].plot([offset[g_id]-W*2,offset[g_id]+w_baseline-1+W],[par_mean_w, par_mean_w],lw=2,ls='--',color=colors[g_id],label='weighted mean rate '+group_str[g_id])        

                    #plot average and std of identifiable parameters:
                    if bool_mean_idPars and par_transformation=='lin':
                        for g_id in range(0,g_end):
                            if opt_comparison == 'H_young_vs_H_aged' or opt_comparison=='H_age_matched_vs_CHIP_vs_MDS':
                                w_baseline=N_indi[g_id]
                            if g_id==0:
                                par_id = DF_1[rate][np.logical_and(DF_1[rate+'_min']<DF_1[rate+'_CI_l'],DF_1[rate+'_max']>DF_1[rate+'_CI_u'])==True]
                            elif g_id==1:
                                par_id = DF_2[rate][np.logical_and(DF_2[rate+'_min']<DF_2[rate+'_CI_l'],DF_2[rate+'_max']>DF_2[rate+'_CI_u'])==True]
                            else:
                                par_id = DF_3[rate][np.logical_and(DF_3[rate+'_min']<DF_3[rate+'_CI_l'],DF_3[rate+'_max']>DF_3[rate+'_CI_u'])==True]
                            if len(par_id)>TH_groups:
                                par_id_mean=np.mean(par_id)
                                par_id_std=np.std(par_id)
                                axes[s_id,col].add_patch(patches.Rectangle((offset[g_id]-W*2, max(DF_1[rate+'_min'][0],par_id_mean-par_id_std*2)), width=w_baseline-1+W*2, height=min((DF_1[rate+'_max'][0]-DF_1[rate+'_min'][0]),4*par_id_std),color=colors[g_id],alpha=0.15,label='mean of idetifiable parameters +/- 2 std '+ group_str[g_id]))          
                                axes[s_id,col].plot([offset[g_id]-W*2,offset[g_id]+w_baseline-1+W],[par_id_mean, par_id_mean],lw=2,ls='--',color=colors[g_id],label='mean of identifiable rates '+group_str[g_id])        
                    
                    #plot 0.025 and 0.975 percentile of estimated posterior distribution based on healthy (without CHIP) samples
                    if opt_comparison=='H_age_matched_vs_CHIP_vs_MDS' or opt_comparison=='H_age_matched_vs_MDS':
                        p_l,p_u,mu_hat = confidence_interval_logNorm_distr_rates(DF_1[rate], [DF_1[rate+'_CI_l'],DF_1[rate+'_CI_u']], 0.05, alpha_errorband, par_transformation,DF_1[rate+'_min'],DF_1[rate+'_max'])
                        w_baseline = np.sum(N_indi)
                        #axes[s_id,col].add_patch(patches.Rectangle((offset[0]-W*2, p_l), width=w_baseline-0.5+W*2, height=(p_u-p_l),color=colors[0],alpha=0.15,label='Common 95% confidence interval '+ group_str[0]))          
                        #axes[s_id,col].plot([offset[0]-W*2,w_baseline-0.5+W*2],[mu_hat, mu_hat],lw=2,ls='--',color=colors[0],label='Common mean of rates group '+group_str[0])    
                        axes[s_id,col].add_patch(patches.Rectangle((offset[0]-W*2, p_l), width=w_baseline-0.5+W*2, height=(p_u-p_l),color='#8E8E8E',alpha=0.15,label='Common 95% confidence interval '+ group_str[0]))          
                        axes[s_id,col].plot([offset[0]-W*2,w_baseline-0.5+W*2],[mu_hat, mu_hat],lw=2,ls='--',color='#8E8E8E',label='Common mean of rates group '+group_str[0])    
                        list_rates_up = []
                        list_rates_down = []
                        if opt_comparison=='H_age_matched_vs_MDS':
                            for idx2 in I_list[1]:
                                if (DF_2[rate].iloc[idx2]>p_u):
                                    list_rates_up.append(DF_2[I_col].iloc[idx2])
                                if (DF_2[rate].iloc[idx2]<p_l):
                                    list_rates_down.append(DF_2[I_col].iloc[idx2])
                        if opt_comparison=='H_age_matched_vs_CHIP_vs_MDS':
                            for idx1 in I_list[0]:
                                if (DF_1[rate].iloc[idx1]>p_u):
                                    list_rates_up.append(DF_1[I_col].iloc[idx1])
                                if (DF_1[rate].iloc[idx1]<p_l):
                                    list_rates_down.append(DF_1[I_col].iloc[idx1])
                            for idx2 in I_list[1]:
                                if (DF_2[rate].iloc[idx2]>p_u):
                                    list_rates_up.append('C'+DF_2[I_col].iloc[idx2][1:])
                                if (DF_2[rate].iloc[idx2]<p_l):
                                    list_rates_down.append('C'+DF_2[I_col].iloc[idx2][1:])
                            for idx3 in I_list[2]:        
                                if (DF_3[rate].iloc[idx3]>p_u):
                                    list_rates_up.append(DF_3[I_col].iloc[idx3])
                                if (DF_3[rate].iloc[idx3]<p_l):
                                    list_rates_down.append(DF_3[I_col].iloc[idx3])
                        if list_rates_up:
                            print(rate +' up-regulated in: '+ ','.join(list_rates_up))
                        if list_rates_down:
                            print(rate+' down-regulated in: '+ ','.join(list_rates_down))
                        if opt_plot_number_deregulated_rates:
                            L.append(list_rates_down)
                            L.append(list_rates_up)
                    #plot CIs:
                    if par_transformation != 'ratio':
                        if not bool_test_par:
                            for I_ID in I_list[0]:
                                H = DF_1[rate+'_CI_u']-DF_1[rate+'_CI_l']
                                axes[s_id,col].add_patch(patches.Rectangle((I_ID - W/2 ,DF_1[rate+'_CI_l'][I_ID]), width=W, height=H[I_ID],color=colors[0],alpha=alpha_val,label='95% confidence interval '+group_str[0] if I_ID==0 else ''))
                            for I_ID in I_list[1]:
                                H = DF_2[rate+'_CI_u']-DF_2[rate+'_CI_l']
                                axes[s_id,col].add_patch(patches.Rectangle((I_ID+offset[1] - W/2 , DF_2[rate+'_CI_l'][I_ID]), width=W, height=H[I_ID],color=colors[1],alpha=alpha_val,label='95% confidence interval '+group_str[1] if I_ID==0 else ''))
                            if opt_comparison=='H_age_matched_vs_CHIP_vs_MDS':
                                for I_ID in I_list[2]:
                                    H = DF_3[rate+'_CI_u']-DF_3[rate+'_CI_l']
                                    axes[s_id,col].add_patch(patches.Rectangle((I_ID+offset[2] - W/2 , DF_3[rate+'_CI_l'][I_ID]), width=W, height=H[I_ID],color=colors[2],alpha=alpha_val,label='95% confidence interval '+group_str[2] if I_ID==0 else ''))
                        else:
                            for I_ID in I_list[0]:
                                H = DF_1[rate+'_CI_u']-DF_1[rate+'_CI_l']
                                axes[s_id,col].add_patch(patches.Rectangle((I_ID - W/2 ,DF_1[rate+'_CI_l'][I_ID]), width=W, height=H[I_ID],color=colors[0],alpha=alpha_val,label='95% confidence interval' if I_ID==0 else ''))
                    #plot opt par:
                    if not bool_test_par:
                        if colors[0]==(0.0,0.0,0.0):
                            axes[s_id,col].plot(I_list[0],DF_1[rate], linestyle='None', marker = 'o', ms=markerSize, color='#FFFFFF', label='Rate estimate '+group_str[0])
                        else:
                            axes[s_id,col].plot(I_list[0],DF_1[rate], linestyle='None', marker = 'o', ms=markerSize, color=colors[0], label='Rate estimate '+group_str[0])
                        axes[s_id,col].plot(offset[1]+I_list[1],DF_2[rate], linestyle='None', marker = 'o', ms=markerSize, color=colors[1], label='Rate estimate '+group_str[1])
                        if opt_comparison=='H_age_matched_vs_CHIP_vs_MDS':
                            axes[s_id,col].plot(offset[2]+I_list[2],DF_3[rate], linestyle='None', marker = 'o', ms=markerSize, color=colors[2], label='Rate estimate '+group_str[2])
                    else:
                        axes[s_id,col].plot(I_list[0],DF_1[rate], linestyle='None', marker = 'o', ms=markerSize, color=colors[0], label='Rate estimate')
                    x_lim = axes[s_id,col].get_xlim()

                    #plot test parameter
                    if bool_test_par and any(DF_1.columns==rate+'_t'):
                        axes[s_id,col].plot(I_list[0],DF_1[rate+'_t'],color='crimson', linestyle='None', marker = 'o', ms=markerSize-5, label='test parameter')

                    # axes limits and labels:
                    # axes[s_id,col].set_xlabel("Sample ID")
                    if par_transformation == 'ratio':
                        axes[s_id,col].set_ylabel('1/' + rate_names[r_id])
                    elif par_transformation == 'lin':
                        axes[s_id,col].set_ylabel(rate_names[r_id])
                    else:
                        axes[s_id,col].set_ylabel(par_transformation + '(' + rate_names[r_id] + ')')
                    #if par_transformation!='lin':
                    #axes[s_id,col].set_ylim(DF_1[rate+'_min'][0], DF_1[rate+'_max'][0])
                    if bool_test_par:
                        axes[s_id,col].set_xlim(x_lim[0]-0.1, x_lim[1]+0.1)
                        axes[s_id,col].set_xlabel('noise level')
                    else:
                        #if s_id==0:
                        #    axes[s_id,col].set_ylim(0,0.15)
                        axes[s_id,col].set_xlim(-1, x_lim[1]+0.1)
                        #axes[s_id,col].set_xlim(-1, len(np.concatenate((I_list[0],I_list[1]+offset[1]),axis=0))+1)
                    #x tick labels
                    if not bool_test_par:
                        if opt_comparison=='H_age_matched_vs_CHIP_vs_MDS':
                            axes[s_id,col].set_xticks(np.concatenate((I_list[0],I_list[1]+offset[1],I_list[2]+offset[2]),axis=0))
                            axes[s_id,col].set_xticklabels(np.concatenate((DF_1[I_col],['C' + x[1:] for x in DF_2[I_col]],DF_3[I_col]),axis=0),rotation=rot_deg,fontsize=28)
                        else:
                            axes[s_id,col].set_xticks(np.concatenate((I_list[0],I_list[1]+offset[1]),axis=0))
                            axes[s_id,col].set_xticklabels(np.concatenate((DF_1[I_col],DF_2[I_col]),axis=0),rotation=rot_deg,fontsize=28)
                    else:
                        axes[s_id,col].set_xticks(I_list[0])
                        axes[s_id,col].set_xticklabels(SSF_str,rotation=rot_deg)

                    if bool_plot_legend==True:
                        if not bool_test_par:
                            s_val = -0.2
                        else:
                            s_val = 1.6
                        #axes[s_id,col].legend(bbox_to_anchor=(s_val, 1),numpoints=1)
                        axes[s_id,col].legend(bbox_to_anchor=(-1,1),numpoints=1)
                        bool_plot_legend=False
                    else: 
                        axes[s_id,col].legend_ = None

                else:   
                    if col_counter not in cols_del:
                        cols_del.append(col_counter)

            if r_id==len(rate_names)-1:
                for i_cd in range(max(cols_del),n_cols-2):
                    f.delaxes(axes[s_id,i_cd])
                if CT[s_id]=='MLP':
                    f.delaxes(axes[s_id,n_cols-1])
            s_id = s_id+1 #states

    # figure settings   
    f.text(0.03, 0.5, 'Model states', va='center', rotation='vertical',fontsize=40)
    if par_transformation == 'ratio':
        string1 = 'Times for Model ' + M_names_opt[sf_id]
    else:
        string1 = 'Rates for Model ' + M_names_opt[sf_id]
    f.text(0.45, 0.95, string1, va='center')
    f.text(x_d, 0.93, 'differentiation', va='center')
    f.text(x_p, 0.93, 'proliferation', va='center')
    f.text(x_a, 0.93, 'apoptosis', va='center')
                
    #save figure    
    if opt_save:
        f.savefig(path_MATLAB_result + '/' + 'ParameterWithCIs_'+ par_transformation + opt_comparison + add_str +'.svg', bbox_inches="tight")
        f.savefig(path_MATLAB_result + '/' + 'ParameterWithCIs_'+ par_transformation + opt_comparison + add_str +'.pdf', bbox_inches="tight")
    
    #plot number of deregulated rates per sample
    if opt_plot_number_deregulated_rates:
        plt = loadPltSettings(20,10) #fonsize, markersize
        if opt_comparison=='H_age_matched_vs_CHIP_vs_MDS':
            plot_number_deregulated_rates_per_sample(DF_1, DF_2, DF_3,I_col,L,I_list,offset,colors,opt_comparison,group_str,path_MATLAB_result,opt_save)
        else:
            plot_number_deregulated_rates_per_sample(DF_1, DF_2, [],I_col,L,I_list,offset,colors,opt_comparison,group_str,path_MATLAB_result,opt_save)
 

In [ ]:
def plot_number_deregulated_rates_per_sample(DF_1, DF_2, DF_3, I_col, L,I_list,offset,colors,opt_comparison,group_str,path_matlab_result,opt_save):
    L=sum(L, [])
    
    plt = loadPltSettings(18,18)
    fig=plt.figure(figsize=(12,6))
    for i in range(0,np.shape(I_list)[0]):
        n_deregulated_rates = []
        if i==0:
            samples = DF_1[I_col]
        elif i==1:
            samples = ['C' + x[1:] for x in DF_2[I_col]]
        else:
            samples = DF_3[I_col]
        for j in range(0,np.shape(I_list[i])[0]):
            n_deregulated_rates.append(L.count(samples[I_list[i][j]]))
        if colors[i]==(0.0,0.0,0.0):
            colors[i]='#FFFFFF'
        #plt.plot(offset[i]+I_list[i],n_deregulated_rates, linestyle='None', marker = 'o', color=colors[i], label='Rate estimate '+group_str[i])
        plt.bar(x=offset[i]+I_list[i],height=n_deregulated_rates,width=0.8,color=colors[i], tick_label='Rate estimate '+group_str[i])
    if i==2:
        samples_all = np.concatenate((I_list[0],I_list[1]+offset[1],I_list[2]+offset[2]),axis=0)
        plt.xticks(samples_all,np.concatenate((DF_1[I_col],['C' + x[1:] for x in DF_2[I_col]],DF_3[I_col]),axis=0),rotation=rot_deg,fontsize=18)
    else:
        samples_all = np.concatenate((I_list[0],I_list[1]+offset[1]),axis=0)
        plt.xticks(samples_all,np.concatenate((DF_1[I_col],DF_2[I_col]),axis=0),rotation=rot_deg,fontsize=20)
    plt.xlabel('Samples')
    plt.ylabel('Number of dereglated parameters')
    plt.xlim([-1,len(samples_all)])
    plt.ylim([-0.5, np.max(n_deregulated_rates)+0.5])
    if opt_save:
        fig.savefig(path_matlab_result + '/' + 'Number_of_deregulated_rates_per_sample.svg', bbox_inches="tight")
        fig.savefig(path_matlab_result + '/' + 'Number_of_deregulated_rates_per_sample.pdf', bbox_inches="tight")

In [5]:
def plotModelFitToData (dir_str_matlab,F_str,SF_str,SSF_str,CT,cols_CT,bool_test,opt_save):
    currentpath = os.getcwd()
    #for f_id in range(0,len(F_str)):
    f_id=0
    for sf_id in range(0,len(SF_str)):
        for ssf_id in range(0,len(SSF_str)):
            path_matlab_result = dir_str_matlab + F_str[f_id] + SF_str[sf_id] + SSF_str[ssf_id] + '/'
            os.chdir(path_matlab_result)
            #get all mat files of interest
            files_sum = [s for s in os.listdir(path_matlab_result) if "ws_modelFitPlot_sum" in s]
            files_i = [s for s in os.listdir(path_matlab_result) if "ws_modelFitPlot" in s]
            files = [s for s in files_i if s not in files_sum]
            for mat_id in range(0,len(files_sum)):
                mat_sum = sio.loadmat(files_sum[mat_id])  
                #print(path_matlab_result)
                mat = sio.loadmat(files[mat_id])  
                ndivs=int(mat['ndivs'][:].flatten())
                [a,b1]=np.shape(mat['S_opt_T'])
                [a,b2]=np.shape(mat_sum['S_opt_T'])
                b=b1+b2
                [r,c]=np.shape(mat['states_str'])
                if mat['states_str'][0][c-1][0]=='D':
                    stopp_threshold = b-2
                else:
                    stopp_threshold = b-1
                n_cols = ndivs+1
                n_rows = np.max(np.shape(mat['states_str']))
                f, axes = plt.subplots(n_rows, n_cols, sharey='row', figsize=(25, 30))
                f.patch.set_facecolor('white')
                f.subplots_adjust(hspace = .25, wspace = 0.35)
                i1=0
                i2=0
                i3=0
                tM1 = pd.Series(mat['t'][:].flatten())/24
                tD1 = pd.Series(mat['tD'][:].flatten())/24
                tM2 = pd.Series(mat_sum['t'][:].flatten())/24
                tD2 = pd.Series(mat_sum['tD'][:].flatten())/24
                for c_id in range(0,n_rows):
                    for d_id in range(0,n_cols):
                        i=i1+i2+i3
                        if i<=stopp_threshold:
                            if d_id<(n_cols-1):
                                YM1 = pd.Series(mat['S_opt_T'][:,i1])
                                if c_id==0 & d_id==1:
                                    print(YM1)
                                if bool_test:
                                    YM1_test = pd.Series(mat['S_test_T'][:,i1])
                                s_down1 = pd.Series(mat['e_down'][:,i1])
                                s_up1 = pd.Series(mat['e_up'][:,i1])
                                YD1 = pd.Series(mat['YD'][:,i1])
                                axes[c_id,d_id].plot(tM1,YM1,color=cols_CT[CT.index(mat['states_str'][0][c_id][0])])
                                axes[c_id,d_id].fill_between(tM1, s_down1, s_up1, alpha=0.2, edgecolor=cols_CT[CT.index(mat['states_str'][0][c_id][0])], facecolor=cols_CT[CT.index(mat['states_str'][0][c_id][0])], linewidth = 2)
                                axes[c_id,d_id].plot(tD1[tD1<8],YD1[tD1<8],color=cols_CT[CT.index(mat['states_str'][0][c_id][0])],marker='o',linestyle='')
                                axes[c_id,d_id].plot(tD1[tD1>=8],YD1[tD1>=8],color=cols_CT[CT.index(mat['states_str'][0][c_id][0])],marker='v',linestyle='')
                                if bool_test:
                                    axes[c_id,d_id].plot(tM1,YM1_test,color=cols_CT[CT.index(mat['states_str'][0][c_id][0])],ls='--')
                                axes[0,d_id].set_title(str(d_id)+' divisions')
                                i1=i1+1
                            else:
                                YM2 = pd.Series(mat_sum['S_opt_T'][:,c_id])
                                if bool_test:
                                    YM2_test = pd.Series(mat_sum['S_test_T'][:,c_id])
                                s_down2 = pd.Series(mat_sum['e_down'][:,c_id])
                                s_up2 = pd.Series(mat_sum['e_up'][:,c_id])
                                YD2 = pd.Series(mat_sum['YD'][:,c_id])
                                axes[c_id,d_id].plot(tM2,YM2,color=cols_CT[CT.index(mat_sum['states_str'][0][c_id][0])])
                                axes[c_id,d_id].fill_between(tM2, s_down2, s_up2, alpha=0.2, edgecolor=cols_CT[CT.index(mat_sum['states_str'][0][c_id][0])], facecolor=cols_CT[CT.index(mat_sum['states_str'][0][c_id][0])], linewidth = 2)
                                axes[c_id,d_id].plot(tD2[tD2<8],YD2[tD2<8],color=cols_CT[CT.index(mat_sum['states_str'][0][c_id][0])],marker='o',linestyle='')
                                axes[c_id,d_id].plot(tD2[tD2>=8],YD2[tD2>=8],color=cols_CT[CT.index(mat_sum['states_str'][0][c_id][0])],marker='v',linestyle='')
                                if bool_test:
                                    axes[c_id,d_id].plot(tM2,YM2_test,color=cols_CT[CT.index(mat_sum['states_str'][0][c_id][0])],ls='--')
                                axes[0,d_id].set_title('sum over divisions')
                                i2=i2+1
                            axes[c_id,d_id].set_xlim([0,8])
                            if c_id==n_rows-1:
                                axes[c_id,d_id].set_xlabel("Days")
                            else:
                                axes[c_id,d_id].get_xaxis().set_ticklabels([''])
                        else:
                            f.delaxes(axes[c_id,d_id])
                            i3=i3+1
                    axes[c_id,0].set_ylabel(mat['states_str'][0][c_id][0])
                f.text(0.04, 0.5, str(mat_sum['trans'][0]), va='center', rotation='vertical')
                f.suptitle(SF_str[sf_id] + SSF_str[ssf_id] + '_' + str(mat['i_repe']) + '_' + str(mat['i_repl']))
                if opt_save:
                    fig = plt.gcf()
                    fig.savefig(path_matlab_result + '/' + 'ModelFit_' + SF_str[sf_id][0:7] + str(mat['i_repe']) + '_' + str(mat['i_repl']) + '.svg', bbox_inches="tight")
                    fig.savefig(path_matlab_result + '/' + 'ModelFit_' + SF_str[sf_id][0:7] + str(mat['i_repe']) + '_' + str(mat['i_repl']) + '.pdf', bbox_inches="tight")
                plt.show()
    os.chdir(currentpath)
    return;

In [ ]:
def confidence_interval_logNorm_distr_rates(r, ci_pl, alpha_ci_pl, alpha, par_transformation, r_min, r_max):
    if par_transformation=='log10':
        rate = np.power(10,r)
        CI_pl = np.power(10,ci_pl)
    elif par_transformation=='ratio':
        rate = np.divide(1,r)
        CI_pl = np.divide(1,ci_pl)
    else:
        rate = r
        CI_pl = ci_pl
    #print(rate)
    #print(CI_pl)
    pdf_hat = random_numbers_logNorm_distr_rates(rate, CI_pl, alpha_ci_pl, 1000)
    CI = np.percentile(pdf_hat,[(alpha/2)*100,(1-alpha/2)*100])

    mean = np.mean(pdf_hat)
    if par_transformation=='log10':
        mean_out = np.log10(mean)
        CI_out = np.sort(np.log10(CI)) 
    elif par_transformation=='ratio':
        mean_out = np.divide(1,mean)
        CI_out = np.sort(np.divide(1,CI))
    else:
        mean_out = mean
        CI_out = CI
    #print(mean_out)
    #print(CI_out)
    #if this CI is outside parameter boundaries --> set them to boundaries
    CI_out[0] = np.max([CI_out[0],np.unique(r_min)[0]])
    CI_out[1] = np.min([CI_out[1],np.unique(r_max)[0]])
    return CI_out[0],CI_out[1],mean_out

In [1]:
def random_numbers_logNorm_distr_rates(r, ci_pl, alpha_ci_pl, n):
    random.seed(1423465)
    z_l = stats.norm.ppf(alpha_ci_pl/2)
    z_u = stats.norm.ppf(1-alpha_ci_pl/2)
    pdf_hat = []
    for idx in range(0,len(r)-1):
    #    s_hat = np.abs(np.log(ci_pl[1][idx]) - np.log(ci_pl[0][idx]))/(-z_l+z_u) 
    #    #print(s_hat)
    #    mu_hat = np.log(r[idx])-(s_hat**2)/2
    #    #print(mu_hat)

        #sigma = np.arange(1,3,dtype=np.float)
        ##solve
        ##(1/2)*(s**2)-z_u*s+np.log(ci[1]/r)==0
        ##(1/2)*(s**2)-z_l*s+np.log(ci[0]/r)==0
        ##sigma[0] = (z_u + np.sqrt((z_u**2)-2*np.log(ci_pl[1][idx]/r[idx])))
        #sigma[0] = (z_u - np.sqrt((z_u**2)-2*np.log(ci_pl[1][idx]/r[idx])))
        #sigma[1] = (z_l + np.sqrt((z_l**2)-2*np.log(ci_pl[0][idx]/r[idx])))
        ##sigma[3] = (z_l - np.sqrt((z_l**2)-2*np.log(ci_pl[0][idx]/r[idx])))
        #if ((z_u**2)-2*np.log(ci_pl[1][idx]/r[idx]))<0:
        #    print("sigma_0 n.d.")
        #if ((z_l**2)-2*np.log(ci_pl[0][idx]/r[idx]))<0:
        #    print("sigma_1 n.d.")
        #s_hat = np.mean(sigma[sigma>0])
        #s_hat_old= (z_u + np.sqrt(z_u**2-2*np.log(ci_pl[1][idx]/r[idx])) + z_l - np.sqrt(z_l**2-2*np.log(ci_pl[0][idx]/r[idx])))/2
        #print('s_hat_old: '+str(s_hat_old))
        #s_hat_old has sign error in formla
        s_hat = (np.log(ci_pl[0][idx])-np.log(ci_pl[1][idx]))/(z_l-z_u)
        #print('s_hat: '+str(s_hat))
        mu_hat = np.log(r[idx])-(s_hat**2)/2
        if idx==0:
            pdf_hat = np.random.lognormal(mu_hat,s_hat,n)
        else:
            pdf_hat = np.concatenate((pdf_hat,np.random.lognormal(mu_hat,s_hat,n)),axis=0)
    
    return pdf_hat

In [7]:
def getScores(dir_str_matlab,F_str,SF_str,SSF_str,Score_label,opt_all,bool_fit_repetitions_seperately):
    if bool_fit_repetitions_seperately:
        add_str = '_rep_sep'
    else:
        add_str = ''
    Scores = np.zeros((len(F_str),len(SF_str),len(SSF_str)))
    Scores_all = np.zeros((len(F_str),len(SF_str)))
    currentpath = os.getcwd()
    f_id=0
    for file_dir in F_str:
        path_matlab_result = dir_str_matlab + file_dir
        os.chdir(path_matlab_result)
        S = sio.loadmat('ws_scores'+add_str+'.mat')  
        if Score_label == 'BIC':
            Scores[f_id,:,:] = S['BIC_mat']
            if opt_all:
                Scores_all[f_id,:] = S['BIC_all_mat'].flatten()
        elif Score_label == 'AIC':
            Scores[f_id,:,:] = S['AIC_mat']
            if opt_all:
                Scores_all[f_id,:] = S['AIC_all_mat'].flatten()
        elif Score_label == 'AIC_c':
            Scores[f_id,:,:] = S['AIC_corrected_mat']
            if opt_all:
                Scores_all[f_id,:] = S['AIC_corrected_all_mat'].flatten()
        elif Score_label == 'LOGL':
            Scores[f_id,:,:] = S['LOGL_mat']
            if opt_all:
                Scores_all[f_id,:] = S['LOGL_all_mat'].flatten()
        f_id=f_id+1
    return Scores, Scores_all

In [8]:
def plot_scores_vs_nis_per_model_and_individual(SSF_str_matlab,model_str,N_is,Scores_all,Score_label,results_str_matlab,bool_all_models,opt_save):
    n_cols = len(SSF_str_matlab)#individuals
    if bool_all_models:
        n_rows = 1
        f, axes = plt.subplots(n_rows, n_cols, figsize=(40, n_rows*2+2))
        f.patch.set_facecolor('white')
        f.subplots_adjust(hspace = .45, wspace = 0.35)
        for m_id in range(0,len(model_str)):
            for i_id in range(0,n_cols):
                axes[i_id].plot(N_is,Scores_all[:,m_id,i_id], linestyle='--', marker='o',color=colors[m_id])
                if m_id==n_rows-1:
                    axes[i_id].set_xlabel("N_iS")
                    axes[i_id].set_title(SSF_str_matlab[i_id][len(SSF_str_matlab[i_id])-3:len(SSF_str_matlab[i_id])])
                if i_id==0:
                    axes[0].set_ylabel(Score_label)    
                axes[n_cols-1].legend(bbox_to_anchor=(1.3, 1),labels=model_str)
        f.text(0.5, 0.99, 'Individuals', va='center')
        fig = plt.gcf()
        fig.savefig(results_str_matlab + '/' + Score_label + '_Scores_vs_Nis_per_individual_all_models.svg', bbox_inches="tight")
        fig.savefig(results_str_matlab + '/' + Score_label + '_Scores_vs_Nis_per_individual_all_models.pdf', bbox_inches="tight")
    else:
        n_rows = len(model_str)#hierarchies
        f, axes = plt.subplots(n_rows, n_cols, figsize=(40,n_rows*3+1))
        f.patch.set_facecolor('white')
        f.subplots_adjust(hspace = .45, wspace = 0.35)
        for m_id in range(0,n_rows):
            for i_id in range(0,n_cols):
                axes[m_id,i_id].plot(N_is,Scores_all[:,m_id,i_id], linestyle='--', marker='o', color='lightseagreen', label=Score_label)
                if m_id==n_rows-1:
                    axes[m_id,i_id].set_xlabel("N_iS")
                if m_id==0:
                    axes[0,i_id].set_title(SSF_str_matlab[i_id][len(SSF_str_matlab[i_id])-3:len(SSF_str_matlab[i_id])])
                if i_id==0:
                    axes[m_id,0].set_ylabel(model_str[m_id]) 
                    if m_id==0:
                        axes[m_id,i_id].legend()
        f.text(0.08, 0.5, Score_label + ' values', va='center', rotation='vertical')
        f.text(0.5, 0.95, 'Individuals', va='center')
        if opt_save:
            fig = plt.gcf()
            fig.savefig(results_str_matlab + '/' + Score_label + '_Scores_vs_Nis_per_model_and_individual.svg', bbox_inches="tight")
            fig.savefig(results_str_matlab + '/' + Score_label + '_Scores_vs_Nis_per_model_and_individual.pdf', bbox_inches="tight")
    return;

In [9]:
def plot_scores_vs_nis_per_model(model_str,N_is,Scores_all,Score_label,path_matlab_result,bool_subplots,opt_save):
    if bool_subplots:
        n_rows = len(model_str)#hierarchies
        n_cols = 1
        f, axes = plt.subplots(n_rows, n_cols, figsize=(7, 15))
        f.subplots_adjust(hspace = .45, wspace = 0.55)
        for m_id in range(0,n_rows):
                axes[m_id].plot(N_is,Scores_all[:,m_id], linestyle='--', marker='o', color='lightseagreen', label=Score_label)
                if m_id==n_rows-1:
                    axes[m_id].set_xlabel("Number of intermediate states")
                axes[m_id].set_ylabel(model_str[m_id]) 
                if m_id==0:
                    axes[m_id].legend()
        f.text(0, 0.5, Score_label + ' values', va='center', rotation='vertical')
        f.patch.set_facecolor('white')
        if opt_save:
            fig = plt.gcf()
            fig.savefig(path_matlab_result + '/' + Score_label + '_Scores_vs_Nis_per_model.svg', bbox_inches="tight")
            fig.savefig(path_matlab_result + '/' + Score_label + '_Scores_vs_Nis_per_model.pdf', bbox_inches="tight")
    else:
        for m_id in range(0,len(model_str)):
            plt.plot(N_is,Scores_all[:,m_id], linestyle='--', marker='o',color=colors[m_id], label=model_str[m_id])
            plt.xlabel("Number of intermediate states")
            plt.ylabel(Score_label + ' value') 
            plt.legend(bbox_to_anchor=(1.3, 1))
        if opt_save:
            fig = plt.gcf()
            fig.savefig(path_matlab_result + '/' + Score_label + '_Scores_vs_Nis_all_models.svg', bbox_inches="tight")
            fig.savefig(path_matlab_result + '/' + Score_label + '_Scores_vs_Nis_all_models.pdf', bbox_inches="tight")
    return;

In [10]:
def plot_scores_vs_individuals_per_model(path_matlab_result,SSF_str_matlab,Scores,Score_label,n_is,N_is,bool_subplots,opt_save):
    idx_nIS_ofInterest = [i for i, n in enumerate(N_is) if n == n_is] 
    if bool_subplots:
        n_rows = 1
        n_cols = len(SSF_str_matlab)#individuals
        f, axes = plt.subplots(n_rows, n_cols, figsize=(40, 7))
        f.subplots_adjust(hspace = .45, wspace = 0.35)
        for i_id in range(0,n_cols):
            for m_id in range(0,len(model_str)):
                axes[i_id].plot(i_id,Scores[idx_nIS_ofInterest[0],m_id,i_id], marker='o',markersize=12,color=colors[m_id])
                if m_id==n_rows-1:
                    axes[i_id].set_xlabel(SSF_str_matlab[i_id])
                if m_id==0:
                    axes[0].set_ylabel(Score_label + ' values')  
                if m_id==len(model_str)-1:
                    axes[n_cols-1].legend(bbox_to_anchor=(1.6, 1),labels=model_str)
        f.text(0.5, 0.95, str(N_is[idx_nIS_ofInterest[0]]) +' intermediate states', va='center')
        f.patch.set_facecolor('white')
        if opt_save:
            fig = plt.gcf()
            fig.savefig(path_matlab_result + '/' + Score_label + '_Scores_vs_Individuals_per_model.svg', bbox_inches="tight")
            fig.savefig(path_matlab_result + '/' + Score_label + '_Scores_vs_Individuals_per_model.pdf', bbox_inches="tight")
    else:
        for m_id in range(0,len(model_str)):
            x=range(0,len(SSF_str_matlab))
            plt.plot(x,Scores[idx_nIS_ofInterest[0],m_id,:].flatten(), marker='o', linestyle = '', color=colors[m_id])
            if m_id==0:
                plt.ylabel(Score_label+' values')  
            if m_id==len(model_str)-1:
                plt.xlabel('Individuals' )
                plt.xticks(x,SSF_str_matlab,rotation=90)
                plt.legend(bbox_to_anchor=(1.6, 1),labels=model_str)
        plt.title(str(N_is[idx_nIS_ofInterest[0]]) +' intermediate states')
        if opt_save:
            plt.savefig(path_matlab_result + '/' + Score_label + '_Scores_vs_Individuals_all_models.svg', bbox_inches="tight")
            plt.savefig(path_matlab_result + '/' + Score_label + '_Scores_vs_Individuals_all_models.pdf', bbox_inches="tight")
    return;

In [11]:
def highlight_small_diffs_to_min(s):
    '''
    highlight other plausible models (difference in BIC to best model <10) in a Series yellow.
    '''
    diff_to_min_below_10 = (s-np.min(s)) < 10
    return ['background-color: orange' if v else '' for v in diff_to_min_below_10 ]

In [12]:
def highlight_min2(s):
    '''
    highlight other plausible models (difference in BIC to best model <10) in a Series yellow.
    '''
    min_bool = (s==s.min())
    return ['background-color: red' if v else '' for v in min_bool ]

In [13]:
def highlight_max2(s):
    '''
    highlight other plausible models (difference in BIC to best model <10) in a Series yellow.
    '''
    min_bool = (s==s.max())
    return ['background-color: red' if v else '' for v in min_bool ]

In [14]:
def get_styled_Score_table(option,N_is,n_is,Scores,Scores_all,Score_label,SSF_str,M_names_opt,bool_fit_repetitions_seperately):
    colNames_list = list()
    if option == 'compare Nis':
        a=1
        df = pd.DataFrame({'model': model_str})
        for ID_IS in N_is:
            idx_nIS_ofInterest = [i for i, n in enumerate(N_is) if n == ID_IS] 
            df[Score_label + '_' + str(ID_IS) + '_iS'] = Scores_all[idx_nIS_ofInterest[0],:]
            colNames_list.append(Score_label + '_' + str(ID_IS) + '_iS')
    elif option == 'compare individuals':
        df = pd.DataFrame({'model': model_str})
        idx_nIS_ofInterest = [i for i, n in enumerate(N_is) if n == n_is] 
        a=0
        for ID_I in range(0,np.shape(Scores)[2]):#len(SSF_str)):
            df[Score_label + '_' + SSF_str[ID_I]] = Scores[idx_nIS_ofInterest[0],:,ID_I]
        if bool_fit_repetitions_seperately:
            df.rename(columns=lambda x: x[len(x)-5:len(x)] if x != 'model' else x, inplace=True)
        else:
            df.rename(columns=lambda x: x[len(x)-3:len(x)] if x != 'model' else x, inplace=True)
        colNames_list = df.columns.str.encode('ascii','ignore')
    elif option == 'compare sim_models to opt_models':
        df = pd.DataFrame({'opt. with\ sim. from': model_str})
        a=0
        #for ID_I in range(0,np.shape(Scores)[2]):
            #print(Score_label + '_' + SSF_str[ID_I])
        for ID_simModel in range(0,np.shape(Scores)[0]):
            df[M_names_opt[ID_simModel]] = np.mean(Scores,axis=2)[ID_simModel,:]
        colNames_list = df.columns
    if Score_label=='LOGL':
        df_styled2 = df.style.apply(highlight_max2, subset=colNames_list[1:len(colNames_list)], axis=a)   
    else:
        df_styled = df.style.apply(highlight_small_diffs_to_min, subset=colNames_list[1:len(colNames_list)], axis=a)
        df_styled2 = df_styled.apply(highlight_min2, subset=colNames_list[1:len(colNames_list)], axis=a)   
    return df_styled2

In [15]:
def calculate_Score_category_percentages(S):
    IDX_min = list()
    IDX_plausible = list()
    N_individuals = np.shape(S)[2]
    N_models = np.shape(S)[1]
    best = list()
    plausible = list()
    rejected = list()
    for i_id in range(0,N_individuals):
        IDX_min = IDX_min + np.where(S == np.min(S[0,0:N_models,i_id]))[1].tolist()
        IDX_plausible = IDX_plausible + np.where((S[0,0:N_models,i_id]-np.min(S[0,0:N_models,i_id])<10) & (S[0,0:N_models,i_id]!=np.min(S[0,0:N_models,i_id])))[0].tolist()
    for m_id in range(0,N_models):
        best.append(100*(IDX_min.count(m_id)/N_individuals))
        plausible.append(100*(IDX_plausible.count(m_id)/N_individuals))
        rejected.append(100*((N_individuals-IDX_min.count(m_id)-IDX_plausible.count(m_id))/N_individuals))
    return best, plausible, rejected

In [16]:
def plot_stackedBarplot_of_Score_category_percentages(best, plausible, rejected, criterion, main_folder_matlab, F_str, str_header,opt_save):
    ind = [M_name for M_name in M_names]

    plt.bar(ind, np.array(rejected), width=0.6, label='rejected', color='grey', bottom=np.array(best)+np.array(plausible))
    plt.bar(ind, np.array(plausible), width=0.6, label='plausible', color='orange', bottom=np.array(best))
    plt.bar(ind, np.array(best), width=0.6, label='best', color='red')

    plt.xticks(ind, M_names)
    plt.ylabel("Percentage")
    plt.xlabel("Models")
    plt.legend(loc="upper right")
    plt.title(criterion + " result for all models based on"+ str_header +"individuals")
    if opt_save:
        fig = plt.gcf()
        fig.savefig(main_folder_matlab + F_str + '/' + criterion + str_header + '_score_category_percentages.svg', bbox_inches="tight")
        fig.savefig(main_folder_matlab + F_str + '/' + criterion + str_header + '_score_category_percentages.pdf', bbox_inches="tight")

    plt.show()

In [17]:
def hex_to_rgb(hex):
    hex = hex.lstrip('#')
    h_length = len(hex)
    rgb_val = tuple(int(hex[i:i+int(h_length/3)], 16)/255 for i in range(0, h_length, int(h_length/3)))
    return rgb_val

In [1]:
def plot_par_vs_age(DF, main_folder_matlab, par_T, F_str, SF_str, model_of_interest, CT, ratenames, colors,opt_comparison,opt_regression_Line,opt_single_regressionLine,bool_plot_IDs,opt_sample_ID,opt_log_transform_rates,opt_plot_CIs,opt_save):
    markerSize=100
    if opt_sample_ID=='short':
        I_col = 'Sample_ID'
    elif opt_sample_ID=='long':
        I_col = 'Sample_ID_long'
    
    bool_plot_legend=True
    bool_mask_empty=False
    bool_mask1_empty=False
    bool_mask2_empty=False
    
    TH_groups = 4
    TH_total = 6 # min of identifiable parameters to draw plot
    
    DF, DF_1, DF_2, lab_str1, lab_str2 = get_dataframe_selection(DF,opt_comparison,opt_sample_ID)
    
    if opt_plot_CIs==True:
        W=0.6
        alpha_val = 0.4
    
    if opt_log_transform_rates:
        DF_log = DF.copy()
        DF_1_log = DF_1.copy()
        DF_2_log = DF_2.copy()
        DF_log[ratenames] = np.log(DF[ratenames])
        DF_1_log[ratenames] = np.log(DF_1[ratenames])
        DF_2_log[ratenames] = np.log(DF_2[ratenames])
        DF_log10 = DF.copy()
        DF_1_log10 = DF_1.copy()
        DF_2_log10 = DF_2.copy()
        DF_log10[ratenames] = np.log10(DF[ratenames])
        DF_1_log10[ratenames] = np.log10(DF_1[ratenames])
        DF_2_log10[ratenames] = np.log10(DF_2[ratenames])
        DF_1_log10[ratenames+'_CI_u'] = np.log10(DF_1[ratenames+'_CI_u'])
        DF_2_log10[ratenames+'_CI_u'] = np.log10(DF_2[ratenames+'_CI_u'])
        DF_1_log10[ratenames+'_CI_l'] = np.log10(DF_1[ratenames+'_CI_l'])
        DF_2_log10[ratenames+'_CI_l'] = np.log10(DF_2[ratenames+'_CI_l'])
        # also transforms CIs
    
    
    
    if model_of_interest=='model_A':
        n_cols = 4
        x_d = 0.26
        x_p = 0.58
        x_a = 0.79
    elif model_of_interest=='model_union_ABDGI':
        n_cols = 5
        x_d = 0.34
        x_p = 0.69
        x_a = 0.84
    else:
        n_cols = 6
        x_d = 0.34
        x_p = 0.69
        x_a = 0.84
    n_rows = len(CT)
    f, axes = plt.subplots(n_rows, n_cols, figsize=(45, 35))
    f.patch.set_facecolor('white')
    f.subplots_adjust(hspace = .45, wspace = 0.3)  
    s_id = 0 #states
    while s_id < len(CT):
        cols_del = list()
        cols_del2 = list()
        col_counter = 0
        for r_id in range(0,len(ratenames)):
            a_m = fnm.fnmatch(ratenames[r_id],'a_'+ CT[s_id] +'*')
            b_m = fnm.fnmatch(ratenames[r_id],'b_'+ CT[s_id] +'*')
            g_m = fnm.fnmatch(ratenames[r_id],'g_'+ CT[s_id] +'*')
            if (a_m or b_m or g_m):
                if a_m:
                    col = col_counter
                    col_counter = col_counter+1
                if b_m:
                    col = n_cols-2
                if g_m:
                    col = n_cols-1
                mask = ~np.isnan(DF[ratenames[r_id]]) 
                mask = ~np.isnan(DF[ratenames[r_id]]) 
                mask_1 = ~np.isnan(DF_1[ratenames[r_id]]) 
                mask_2 = ~np.isnan(DF_2[ratenames[r_id]]) 
                mask_ids = DF[mask].index.values.astype(int)
                mask_ids_1 = DF_1[mask_1].index.values.astype(int)
                mask_ids_2 = DF_2[mask_2].index.values.astype(int)

                if mask_ids_1.size >= TH_groups and mask_ids_1.size+mask_ids_2.size >= TH_total:
                    bool_mask1_empty=False
                    if opt_log_transform_rates:
                        DF_1_log10.iloc[mask_ids_1].plot('age',ratenames[r_id],kind='scatter',s=markerSize,marker='o',color=colors[0],ax=axes[s_id,col],label='Rate estimate ' +lab_str1)
                        if par_T != 'ratio' and opt_plot_CIs == True :
                            CI_u = DF_1_log10[ratenames[r_id]+'_CI_u'].iloc[mask_ids_1]
                            CI_l = DF_1_log10[ratenames[r_id]+'_CI_l'].iloc[mask_ids_1]
                            H = CI_u-CI_l
                            for age_id, age in enumerate(DF_1_log10['age'].iloc[mask_ids_1]):
                                axes[s_id,col].add_patch(patches.Rectangle((age - W/2 ,CI_l.iloc[age_id]), width=W, height=H.iloc[age_id],color=colors[0],alpha=alpha_val,label='95% confidence interval '+lab_str1 if age_id == 0 else ''))
                    else:
                        DF_1.iloc[mask_ids_1].plot('age',ratenames[r_id],kind='scatter',s=markerSize,marker='o',color=colors[0],ax=axes[s_id,col],label='Rate estimate '+lab_str1)
                        if par_T != 'ratio' and opt_plot_CIs == True :
                            CI_u = DF_1[ratenames[r_id]+'_CI_u'].iloc[mask_ids_1]
                            CI_l = DF_1[ratenames[r_id]+'_CI_l'].iloc[mask_ids_1]
                            H = CI_u-CI_l
                            for age_id, age in enumerate(DF_1['age'].iloc[mask_ids_1]):
                           #     if age_id==0:
                           #         print(CI_u[0])
                           #         print(CI_l[0])
                           #         print(H[0])
                           #         print(DF_1[ratenames[r_id]].iloc[mask_ids_1[0]])
                                axes[s_id,col].add_patch(patches.Rectangle((age - W/2 ,CI_l.iloc[age_id]), width=W, height=H.iloc[age_id],color=colors[0],alpha=alpha_val,label='95% confidence interval '+lab_str1 if age_id == 0 else ''))
                else:
                    bool_mask1_empty=True

                if mask_ids_2.size >= TH_groups and mask_ids_1.size+mask_ids_2.size >= TH_total:
                    bool_mask2_empty=False
                    if opt_log_transform_rates:
                        #print(DF_2_log10[['age',ratenames[r_id]]])  
                        DF_2_log10.iloc[mask_ids_2].plot('age',ratenames[r_id],kind='scatter',s=markerSize,marker='o',color=colors[1],ax=axes[s_id,col],label='Rate estimate ' +lab_str2)
                        if par_T != 'ratio' and opt_plot_CIs == True:
                            CI_u = DF_2_log10[ratenames[r_id]+'_CI_u'].iloc[mask_ids_2]
                            CI_l = DF_2_log10[ratenames[r_id]+'_CI_l'].iloc[mask_ids_2]
                            H = CI_u-CI_l
                            for age_id, age in enumerate(DF_2_log10['age'].iloc[mask_ids_2]):
                                axes[s_id,col].add_patch(patches.Rectangle((age - W/2 , CI_l.iloc[age_id]), width=W, height=H.iloc[age_id],color=colors[1],alpha=alpha_val,label='95% confidence interval '+lab_str2 if age_id == 0 else ''))
                    else:
                        DF_2.iloc[mask_ids_2].plot('age',ratenames[r_id],kind='scatter',s=markerSize,marker='o',color=colors[1],ax=axes[s_id,col],label='Rate estimate ' +lab_str2)
                        if par_T != 'ratio' and opt_plot_CIs == True:
                            CI_u = DF_2[ratenames[r_id]+'_CI_u'].iloc[mask_ids_2]
                            CI_l = DF_2[ratenames[r_id]+'_CI_l'].iloc[mask_ids_2]
                            H = CI_u - CI_l
                            R = DF_2[ratenames[r_id]].iloc[mask_ids_2]
                            for age_id, age in enumerate(DF_2['age'].iloc[mask_ids_2]):
                                #if age_id==0:
                                #print(ratenames[r_id])
                                #print(age)
                                #print(CI_u.iloc[age_id])
                                #print(CI_l.iloc[age_id])
                                #print(H.iloc[age_id])
                                #print(R.iloc[age_id])
                                axes[s_id,col].add_patch(patches.Rectangle((age - W/2 , CI_l.iloc[age_id]), width=W, height=H.iloc[age_id],color=colors[1],alpha=alpha_val,label='95% confidence interval '+lab_str2 if age_id == 0 else ''))

                    if bool_plot_IDs:
                        for m_id in mask_ids_2:
                            axes[s_id,col].text(DF_2['age'][m_id],DF_2[ratenames[r_id]][m_id],DF_2['Sample_ID'][m_id])
                else:
                    bool_mask2_empty=True

                if bool_plot_IDs:
                    for m_id in mask_ids_1:
                        axes[s_id,col].text(DF_1['age'][m_id],DF_1[ratenames[r_id]][m_id],DF_1['Sample_ID'][m_id])
                        axes[s_id,col].text(DF_1['age'][m_id],DF_1[ratenames[r_id]][m_id],DF_2['Sample_ID'][m_id])
                if opt_regression_Line:
                    if opt_single_regressionLine:
                        if mask_ids.size >= TH_total:
                            bool_mask_empty=False
                            if opt_log_transform_rates:
                                gradient, intercept, r_value, p_value, std_err = stats.linregress(DF_log['age'][mask],DF_log[ratenames[r_id]][mask])
                                axes[s_id,col].plot(DF_log10['age'][mask],np.log10(np.exp(gradient*DF['age'][mask]+intercept)),color=hex_to_rgb('#404040'))
                                if p_value<=0.05:
                                    axes[s_id,col].text(axes[s_id,col].get_xlim()[0]+10,axes[s_id,col].get_ylim()[1],'p='+str(round(p_value,3)),color=hex_to_rgb('#404040'),label='regression line ')
                            else:    
                                gradient, intercept, r_value, p_value, std_err = stats.linregress(DF['age'][mask],DF[ratenames[r_id]][mask])
                                axes[s_id,col].plot(DF['age'][mask],gradient*DF['age'][mask]+intercept,color=hex_to_rgb('#404040'))
                                if p_value<=0.05:
                                    axes[s_id,col].text(axes[s_id,col].get_xlim()[0]+10,axes[s_id,col].get_ylim()[1],'p='+str(round(p_value,3)),color=hex_to_rgb('#404040'),label='regression line ')
                                    #axes[s_id,col].text(np.min(DF['age'][mask]),np.min(DF[ratenames[r_id]][mask]),'p='+str(round(p_value,3)),color=hex_to_rgb('#404040'),label='regression line')
                        else:
                            bool_mask_empty=True
                    else:
                        if mask_ids_1.size >= TH_groups and mask_ids_1.size+mask_ids_2.size >= TH_total:
                            bool_mask1_empty=False
                            if opt_log_transform_rates:
                                gradient_1, intercept_1, r_value_1, p_value_1, std_err_1 = stats.linregress(DF_1_log['age'][mask_1],DF_1_log[ratenames[r_id]][mask_1])
                                axes[s_id,col].plot(DF_1_log10['age'][mask_1],np.log10(np.exp(gradient_1*DF_1_log10['age'][mask_1]+intercept_1)),color=colors[0],label='regression line '+lab_str1)
                            else:
                                gradient_1, intercept_1, r_value_1, p_value_1, std_err_1 = stats.linregress(DF_1['age'][mask_1],DF_1[ratenames[r_id]][mask_1])
                                axes[s_id,col].plot(DF_1['age'][mask_1],gradient_1*DF_1['age'][mask_1]+intercept_1,color=colors[0],label='regression line '+lab_str1)
                            if p_value_1<=0.05:
                                axes[s_id,col].text(axes[s_id,col].get_xlim()[0]+10,axes[s_id,col].get_ylim()[1],'p='+str(round(p_value_1,3)),color=colors[0])
                        if mask_ids_2.size >= TH_groups and mask_ids_1.size+mask_ids_2.size >= TH_total:
                            bool_mask2_empty=False
                            if opt_log_transform_rates:
                                gradient_2, intercept_2, r_value_2, p_value_2, std_err_2 = stats.linregress(DF_2_log['age'][mask_2],DF_2_log[ratenames[r_id]][mask_2])
                                axes[s_id,col].plot(DF_2_log10['age'][mask_2],np.log10(np.exp(gradient_2*DF_2_log10['age'][mask_2]+intercept_2)),color=colors[1],label='regression line '+lab_str2)
                            else:
                                gradient_2, intercept_2, r_value_2, p_value_2, std_err_2 = stats.linregress(DF_2['age'][mask_2],DF_2[ratenames[r_id]][mask_2])
                                axes[s_id,col].plot(DF_2['age'][mask_2],gradient_2*DF_2['age'][mask_2]+intercept_2,color=colors[1],label='regression line '+lab_str2)
                            if p_value_2<=0.05:
                                axes[s_id,col].text(axes[s_id,col].get_xlim()[1]-25,axes[s_id,col].get_ylim()[1],'p='+str(round(p_value_2,3)),color=colors[1])       
                # axes limits and labels:
                axes[s_id,col].set_xlim(min(DF['age'])-1,max(DF['age'])+1)
                a,b = axes[s_id,col].get_ylim()
                if opt_log_transform_rates==False:
                    axes[s_id,col].set_ylim(0,b)
                #axes[s_id,col].set_ylim(0.01,0.06)
                axes[s_id,col].set_xlabel("age")
                axes[s_id,col].set_ylabel(ratenames[r_id])
                if opt_log_transform_rates:
                    yloc = axes[s_id,col].get_yticks()
                    axes[s_id,col].set_yticklabels(np.round(10**yloc,decimals=3))
                #    axes[s_id,col].set_ylabel('log10( ' + ratenames[r_id]+ ')')
                if bool_mask_empty or (bool_mask1_empty and bool_mask2_empty):
                    cols_del2.append(col)
                else:
                    if bool_plot_legend==True:
                        axes[s_id,col].legend(bbox_to_anchor=(-0.2, 1),scatterpoints=1)
                        bool_plot_legend=False
                    else: 
                        axes[s_id,col].get_legend().set_visible(False)
            else:
                if col_counter not in cols_del:
                    cols_del.append(col_counter)
        if r_id==len(ratenames)-1:
            for i_cd in range(max(cols_del),n_cols-2):
                f.delaxes(axes[s_id,i_cd])
            if CT[s_id]=='MLP':
                f.delaxes(axes[s_id,n_cols-1])
            for i_cd in cols_del2:
                f.delaxes(axes[s_id,i_cd])
        s_id = s_id+1 #states
        col_counter = 0
    f.text(0.07, 0.5, 'Model states', va='center', rotation='vertical')
    string1 = 'Rates for Model: ' + model_of_interest
    f.text(0.45, 0.95, string1, va='center')
    f.text(x_d, 0.93, 'differentiation', va='center')
    f.text(x_p, 0.93, 'proliferation', va='center')
    f.text(x_a, 0.93, 'apoptosis', va='center')
    if opt_single_regressionLine:
        add_str = '_singeRegrLine'
    else:
        add_str = '_2regLines'
    if bool_plot_IDs:
        add_str2 = '_withIDs'
    else:
        add_str2 =''
    if opt_save:
        fig = plt.gcf()
        fig.savefig(main_folder_matlab + F_str[0] + SF_str[0] + '/' + 'Parameter_' + par_T +'_vs_age_' + opt_comparison + add_str + add_str2 + '.svg', bbox_inches="tight")
        fig.savefig(main_folder_matlab + F_str[0] + SF_str[0] + '/' + 'Parameter_' + par_T +'_vs_age_' + opt_comparison + add_str + add_str2 + '.pdf', bbox_inches="tight")
    plt.show()

In [19]:
def get_mean_and_meadian_of_Par_Result(DF,ratenames,model_of_interest):
    data_opt=list()
    data_opt_y_a=list()
    for r_id in range(0,len(ratenames)):
        DF_y = DF[DF['age']<=55].copy()
        DF_o = DF[DF['age']>55].copy()
        data_opt.append([model_of_interest,ratenames[r_id],np.nanmedian(DF[ratenames[r_id]]),np.nanmean(DF[ratenames[r_id]]),np.nanstd(DF[ratenames[r_id]])])
        data_opt_y_a.append([model_of_interest,ratenames[r_id],np.nanmedian(DF_y[ratenames[r_id]]),np.nanmean(DF_y[ratenames[r_id]]),np.nanstd(DF_y[ratenames[r_id]]),np.nanmedian(DF_o[ratenames[r_id]]),np.nanmean(DF_o[ratenames[r_id]]),np.nanstd(DF_o[ratenames[r_id]])])
    df_opt_par = pd.DataFrame(data_opt, columns = ['model','parameter', 'median', 'mean', 'std'])
    df_opt_par_y_a = pd.DataFrame(data_opt_y_a, columns = ['model','parameter', 'median_y','mean_y','std_y', 'median_a', 'mean_a','std_a'])
    return df_opt_par, df_opt_par_y_a

In [1]:
def getValues4PCAplot(comparison_str,DF,opt_sample_ID):
    
    if opt_sample_ID=='short':
        I_col = 'Sample_ID'
    elif opt_sample_ID=='long':
        I_col = 'Sample_ID_long'
        
    a,b = np.shape(DF)
    y=list()
    s=list()
    
    yellow = '#F9EC39'
    white = '#FFFFFF'
    violet='#7A3DB7'
    orange='#FF8000'
    firered='#FF0000'
    darkred='#8B1313'
    mutation_str = DF.columns[DF.columns.str.endswith('VAF')]
    if comparison_str=='Disease status MDS':
        groups = ('MDS','healthy age-matched','CHIP')
        groups_colors = [('MDS', col_MDS),('healthy age-matched', col_H_a),('CHIP', col_CHIP)]
        for i in range(0,a):
            if DF[I_col][i].startswith('MDS'):
                y.append(groups[0])
                s.append(DF[I_col][i])
            else:
                if np.any(DF[mutation_str].values.astype(np.float)>0,axis=1)[i]==False:
                    y.append(groups[2]) # CHIP
                    sample_id = DF[I_col][i] 
                    sample_id = 'C' + sample_id[1:]
                    s.append(sample_id)
                else:
                    y.append(groups[1])   
                    s.append(DF[I_col][i])
    elif comparison_str=='Disease status CHIP':
        groups = ('CHIP','healthy age-matched','MDS')
        groups_colors = [('CHIP', col_CHIP),('healthy age-matched', col_H_a),('MDS', col_MDS)]
        for i in range(0,a):
            s.append(DF[I_col][i])
            if DF[I_col][i].startswith('MDS'):
                y.append(groups[2])
            else:
                if np.any(DF[mutation_str].values.astype(np.float)>0,axis=1)[i]==False:
                    y.append(groups[1]) # healthy non-CHIP
                else:
                    y.append(groups[0])
    elif comparison_str == 'Disease status healthy':
        groups = ('healthy age-matched','CHIP','MDS')
        groups_colors = [('healthy age-matched', col_H_a),('CHIP', col_CHIP),('MDS', col_MDS)]
        for i in range(0,a):
            s.append(DF[I_col][i])
            if DF[I_col][i].startswith('MDS'):
                y.append(groups[2])
            else:
                if np.any(DF[mutation_str].values.astype(np.float)>0,axis=1)[i]==False:
                    y.append(groups[0]) # healthy non-CHIP
                else:
                    y.append(groups[1])
    elif comparison_str == 'BM sample type':
        groups=DF['sample_type'].unique().tolist()
        print(groups)
        if len(groups)==2:  
            groups_colors=[(groups[0],'#009999'),(groups[1],'#99004C')]
        else:
            groups_colors=[(groups[0],'#009999'),(groups[1],'#99004C'),(groups[2],'#CC6600')]
        for i in range(0,a):
            for g_id in range(0,len(groups)):
                s.append(DF[I_col][i])
                if DF['sample_type'][i]==groups[g_id]: 
                    y.append(groups[g_id])
    elif comparison_str == 'Gender':
        groups=('male','female')
        groups_colors=[('male','#004C99'),('female','#00CCCC')]      
        for i in range(0,a):
            s.append(DF[I_col][i])
            if DF['sex'][i].startswith('w') or DF['sex'][i].startswith('f'): 
                y.append(groups[1])
            else: 
                y.append(groups[0])   
    elif comparison_str == 'Number of mutations':
        groups=('0','1','2','3','>3')
        groups_colors=[('0',col_H_a),('1',yellow),('2','#FF8000'),('3','#FF0000'),('>3','#990000')]
        for i in range(0,a):
            s.append(DF[I_col][i])
            if DF['Mutation_counts'][i]==0: 
                y.append(groups[0])
            elif DF['Mutation_counts'][i]==1: 
                y.append(groups[1])
            elif DF['Mutation_counts'][i]==2: 
                y.append(groups[2])
            elif DF['Mutation_counts'][i]==3: 
                y.append(groups[3])
            else: 
                y.append(groups[4])  
    elif comparison_str == 'SF3B1 or sideroblasts':
        groups=('SF3B1','sideroblasts','both','none')
        groups_colors=[('SF3B1','#FF8000'),('sideroblasts','#FF0000'),('both','#004C99'),('none',white)]
        for i in range(0,a):
            s.append(DF[I_col][i])
            if np.logical_and(isinstance(DF['SF3B1'][i],str),DF['SF3B1'][i]!='no'): 
                if np.logical_and(isinstance(DF['sideroblasts'][i],str),DF['sideroblasts'][i]!='no'): 
                    y.append(groups[2]) 
                else: 
                    y.append(groups[0])  
            elif np.logical_and(isinstance(DF['sideroblasts'][i],str),DF['sideroblasts'][i]!='no'): 
                y.append(groups[1])  
            else:
                y.append(groups[3])
    elif comparison_str == 'Leucocyte level':
        groups=('below normal','normal','increased')
        groups_colors=[('below normal','#FF8000'),('normal',white),('increased','#CC0000')]
        for i in range(0,a):
            s.append(DF[I_col][i])
            if isinstance(DF['leucocytes'][i],str):
                num_list = re.findall("[+-]?\d+\.\d+", DF['leucocytes'][i])
                if float(num_list[0])<4:
                    y.append(groups[0]) 
                elif float(num_list[0])>10:
                    y.append(groups[2]) 
                else:
                    y.append(groups[1]) 
            else:
                y.append(groups[1]) 
    elif comparison_str == 'Platelet level':
        groups=('below normal','normal','increased')
        groups_colors=[('below normal','#FF8000'),('normal',white),('increased','#CC0000')]
        for i in range(0,a):
            s.append(DF[I_col][i])
            if isinstance(DF['thrombocytes'][i],str):
                num_list = re.findall(r'\d+', DF['thrombocytes'][i])
                if num_list:
                    if float(num_list[0])<150:
                        y.append(groups[0]) 
                    elif float(num_list[0])>350:
                        y.append(groups[2]) 
                    else:
                        y.append(groups[1]) 
                else:
                    y.append(groups[1]) 
            else:
                y.append(groups[1]) 
    elif comparison_str == 'Hemoglobine level':
        groups=('below normal','normal','increased')
        groups_colors=[('below normal','#FF8000'),('normal',white),('increased','#CC0000')]
        for i in range(0,a):
            s.append(DF[I_col][i])
            if isinstance(DF['hemoglobine'][i],str):
                if DF['sex'][i].startswith('m'):
                    TH_min = 13.5
                    TH_max = 17
                else:
                    TH_min = 12
                    TH_max = 16
                num_list = re.findall("[+-]?\d+\.\d+", DF['hemoglobine'][i])
                if num_list:
                    if float(num_list[0])<TH_min:
                        y.append(groups[0]) 
                    elif float(num_list[0])>TH_max:
                        y.append(groups[2]) 
                    else:
                        y.append(groups[1]) 
                else:
                    y.append(groups[1]) 
            else:
                y.append(groups[1]) 
    elif comparison_str == 'WHO classification':
        groups=('healthy','CHIP','MDS-RS-MLD','MDS-EB1','MDS-EB2','CMML-2','MDS/MPN-RS-T')
        groups_colors=[('healthy',white),('CHIP',col_CHIP),('MDS-RS-MLD',violet),('MDS-EB1',orange),('MDS-EB2',firered),('CMML-2',darkred),('MDS/MPN-RS-T',yellow)]
        for i in range(0,a):
            s.append(DF[I_col][i])
            search_str = DF['diagnosis_and_WHO_subtype'][i].replace(" ", "")
            if search_str=='healthy':
                y.append(groups[0]) 
            elif search_str=='CHIP':
                y.append(groups[1])
            elif search_str=='MDS-RS-MLD':
                y.append(groups[2]) 
            elif search_str=='MDS-EB1':
                y.append(groups[3]) 
            elif search_str=='MDS-EB2':
                y.append(groups[4]) 
            elif search_str=='CMML-2':
                y.append(groups[5]) 
            elif search_str=='MDS/MPN-RS-T':
                y.append(groups[6]) 
    elif comparison_str == 'IPSS-R Score':
        groups=('healthy','low risk','intermediate risk','high risk','NaN')
        groups_colors=[('healthy',white),('low risk',yellow),('intermediate risk','#FF8000'),('high risk','#FF0000'),('NaN','#808080')]
        for i in range(0,a):
            s.append(DF[I_col][i])
            if DF[I_col][i].startswith('H'):
                y.append('healthy')
            elif isinstance(DF['IPSS_R'][i],str):
                num_list_int = re.findall(r'\d+', DF['IPSS_R'][i])
                num_list_float = re.findall("[+-]?\d+\.\d+", DF['IPSS_R'][i])
                score = []
                if num_list_int and num_list_float:
                    score = np.max([float(num_list_int[0]),float(num_list_float[0])])
                elif num_list_int:
                    score = float(num_list_int[0])
                elif num_list_float:
                    score = float(num_list_float[0])
                if score:
                    if score<=3:
                        y.append('low risk')
                    elif score>3 and score<=4.5:
                        y.append('intermediate risk')
                    else:
                        y.append('high risk')
                else:
                    y.append('NaN')    
            else:
                y.append('NaN')  
    elif comparison_str == 'Blast percentage':
        groups=('NaN','<5%','5-10%','>10%')
        groups_colors=[('NaN','#194795'),('<5%','#00CC66'),('5-10%','#FF8000'),('>10%','#CC0000')]
        for i in range(0,a):
            s.append(DF[I_col][i])
            if isinstance(DF['blast_percentage'][i],str): #not NaN
                num_list_int = re.findall(r'\d+',DF['blast_percentage'][i])
                num_list_float = re.findall("[+-]?\d+\.\d+", DF['blast_percentage'][i])
                score = []
                if num_list_int and num_list_float:
                    score = np.max([float(num_list_int[0]),float(num_list_float[0])])
                elif num_list_int:
                    score = float(num_list_int[0])
                elif num_list_float:
                    score = float(num_list_float[0])
                if score:
                    if score<=5:
                        y.append('<5%')
                    elif score>5 and score<=10:
                        y.append('5-10%')
                    else:
                        y.append('>10%')
                else:
                    y.append('NaN')    
            else:
                y.append('NaN')          
    elif comparison_str == 'Karyotype':
        groups=('normal','abnormal')
        groups_colors=[('normal','#00CC66'),('abnormal','#CC0000')]
        for i in range(0,a):
            s.append(DF[I_col][i])
            if isinstance(DF['karyotype'][i],str): #not NaN
                if DF['karyotype'][i]=='46 XY [20]' or DF['karyotype'][i]=='46 XX [20]':
                    y.append('normal')
                else:
                    y.append('abnormal')
            else:
                y.append('normal')
    #elif comparison_str =='parameter result':
    #    for i in range(0,a):
    #        s.append(DF[I_col][i])
    #        if np.logical_or(np.logical_or(DF[I_col][i].startswith('H'),DF[I_col][i].endswith('227')),DF[I_col][i].endswith('373')):
    #            y.append('normal')
    #        elif np.logical_or(DF[I_col][i]=='MDS326',DF[I_col][i]=='MDS354'):
    #            y.append('increased proliferation')
    #        else:
    #            y.append('decreased progenitor proliferation')
    return s,y,groups,groups_colors

In [1]:
def confidence_ellipse(x, y, ax, f_color, alpha_val, W, confidence_levels):

    if x.size != y.size:
        raise ValueError("x and y must be the same size")
    
    if W.size>0:
        cov = np.cov(x, y, aweights = W.flatten())
        # Calculating the stdandard deviation of x from
        # the squareroot of the variance and multiplying
        # with the given number of standard deviations.
        mean_x = np.average(x,weights = W.transpose())
        # calculating the stdandard deviation of y ...
        mean_y = np.average(y,weights = W.transpose())
    else:
        cov = np.cov(x, y)
        # Calculating the stdandard deviation of x from
        # the squareroot of the variance and multiplying
        # with the given number of standard deviations.
        mean_x = np.mean(x)
        # calculating the stdandard deviation of y ...
        mean_y = np.mean(y)
        
    eig_vals, eig_vecs = np.linalg.eig(cov)
    eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]
    # Sort the (eigenvalue, eigenvector) tuples from high to low
    eig_pairs.sort()
    eig_pairs.reverse()
    alpha = math.degrees(np.arctan(eig_pairs[0][1][1]/eig_pairs[0][1][0]))
    ## Using a special case to obtain the eigenvalues of this
    ## two-dimensionl dataset.
    for cl in confidence_levels:
        s=chi2.isf(q=1-cl, df=2)
        ell_radius_x = np.sqrt(s*eig_pairs[0][0])
        ell_radius_y = np.sqrt(s*eig_pairs[1][0])
        ellipse = Ellipse((mean_x, mean_y),
            width=2*ell_radius_x,
            height=2*ell_radius_y,
            angle = alpha,
            fc=f_color,
            edgecolor = 'black',
            linestyle = '--',
            alpha=alpha_val)
        ax.add_patch(ellipse)
    return ax

In [2]:
def plot_PCA_2D_projection(D_str,F_str,SF_str,Y,Y_H_1,y_H_2,var_exp,comparison_str,bool_plotIDs,IDX_PCs,add_file_str,legend_loc,W_H_1,W_H_2,cl,opt_ellipse_groups,opt_save):#,n_cols,n_rows):
    
    if np.size(comparison_str)>1 or np.size(IDX_PCs)>2:
        n_rows = np.size(comparison_str)
        n_cols = np.shape(IDX_PCs)[0]
        fig, axis = plt.subplots(n_rows, n_cols, figsize=(n_cols*12, n_rows*6))
        #fig, axis = plt.subplots(n_rows, n_cols, figsize=(45, 70))
        fig.patch.set_facecolor('white')
        fig.subplots_adjust(hspace = 0.2, wspace = 0.2) 
        for rid in range(0,n_rows):
            if np.size(comparison_str)==1:
                s,y,groups,groups_colors = getValues4PCAplot(comparison_str,DF,opt_sample_ID)
            else:
                s,y,groups,groups_colors = getValues4PCAplot(comparison_str[rid],DF,opt_sample_ID)
            for cid in range(0,n_cols):
                #draw ellipse for healthy population:
                #axis[rid,cid] = plt.gca()
                if 1 in opt_ellipse_groups:
                    confidence_ellipse(x=Y_H_1[:,IDX_PCs[cid][0]].flatten(), y=Y_H_1[:,IDX_PCs[cid][1]].flatten(), ax=axis[rid,cid], f_color='#194795', alpha_val=0.15, W=W_H_1, confidence_levels = cl)    
                if 2 in opt_ellipse_groups:
                    confidence_ellipse(x=Y_H_2[:,IDX_PCs[cid][0]].flatten(), y=Y_H_2[:,IDX_PCs[cid][1]].flatten(), ax=axis[rid,cid], f_color='#009999', alpha_val=0.15, W=W_H_2, confidence_levels = cl)
                c,d = np.shape(Y)
                #for id in range(0,np.shape(y)):
                bool_first = np.ones((len(groups),1), dtype=bool)
                for name, col in groups_colors:
                    for i in range(0,c):
                        if y[i]==name:
                            g_id = groups.index(name)
                            axis[rid,cid].scatter(Y[i,IDX_PCs[cid][0]],Y[i,IDX_PCs[cid][1]],color = col, s=90, edgecolors='#010101',zorder=2, label=name if bool_first[g_id] else "")  
                            if (bool_plotIDs and rid==0):
                                axis[rid,cid].text(Y[i,IDX_PCs[cid][0]],Y[i,IDX_PCs[cid][1]],s[i],fontsize=13)
                            #axes[r_id,c_id] = plt.scatter(Y[i,0],Y[i,1],color = col, label=name if bool_first[g_id] else "")  
                            bool_first[g_id] = False
                #plt.setp(axis.get_xticklabels(), visible=False)
                #plt.setp(axis.get_yticklabels(), visible=False)
                axis[rid,cid].set_xlabel('PC'+str(IDX_PCs[cid][0]+1)+' ('+str(round(var_exp[IDX_PCs[cid][0]],2))+'%)')
                axis[rid,cid].set_ylabel('PC'+str(IDX_PCs[cid][1]+1)+' ('+str(round(var_exp[IDX_PCs[cid][1]],2))+'%)')
                if cid==0:
                    lg = axis[rid,cid].legend(title=comparison_str[rid], loc=legend_loc, scatterpoints=1, fontsize=15)
                    lg.get_title().set_fontsize(15)
        filename_str = '_all'
    else:
        s,y,groups,groups_colors = getValues4PCAplot(comparison_str,DF,opt_sample_ID)
        #f, axes = plt.subplots(n_rows, n_cols, figsize=(45, 35))
        #f.patch.set_facecolor('white')
        #f.subplots_adjust(hspace = .45, wspace = 0.2) 
        #r_id=0
        #c_id=0
        fig=plt.figure(figsize=(10,7))
        #draw ellipse for healthy population:
        axis = plt.gca()
        if 1 in opt_ellipse_groups:
            confidence_ellipse(x=Y_H_1[:,IDX_PCs[0]].flatten(), y=Y_H_1[:,IDX_PCs[1]].flatten(), ax=axis, f_color='#194795', alpha_val=0.15, W=W_H_1, confidence_levels = cl)    
        if 2 in opt_ellipse_groups:
            confidence_ellipse(x=Y_H_2[:,IDX_PCs[0]].flatten(), y=Y_H_2[:,IDX_PCs[1]].flatten(), ax=axis, f_color='#009999', alpha_val=0.15, W=W_H_2, confidence_levels = cl)
        c,d = np.shape(Y)
        #for id in range(0,np.shape(y)):
        bool_first = np.ones((len(groups),1), dtype=bool)
        for name, col in groups_colors:
            for i in range(0,c):
                if y[i]==name:
                    g_id = groups.index(name)
                    axis.scatter(Y[i,IDX_PCs[0]],Y[i,IDX_PCs[1]],color = col, s=90, edgecolors='#010101', label=name if bool_first[g_id] else "")  
                    if bool_plotIDs:
                        axis.text(Y[i,IDX_PCs[0]],Y[i,IDX_PCs[1]],s[i],fontsize=10)
                    #axes[r_id,c_id] = plt.scatter(Y[i,0],Y[i,1],color = col, label=name if bool_first[g_id] else "")  
                    bool_first[g_id] = False
        #plt.setp(axis.get_xticklabels(), visible=False)
        #plt.setp(axis.get_yticklabels(), visible=False)
        plt.title('PCA: '+comparison_str)
        plt.xlabel('PC'+str(IDX_PCs[0]+1)+' ('+str(round(var_exp[IDX_PCs[0]],2))+'%)')
        plt.ylabel('PC'+str(IDX_PCs[1]+1)+' ('+str(round(var_exp[IDX_PCs[1]],2))+'%)')
        plt.legend(loc=legend_loc, scatterpoints=1)
        filename_str = str(IDX_PCs[0]+1) +'_vs_PC'+ str(IDX_PCs[1]+1) + '_' + comparison_str
    if bool_plotIDs:
        add_file_str2 = '_withIDs'
    else:
        add_file_str2 = ''
    
    if opt_save:
        fig = plt.gcf() 
        fig.savefig(D_str + F_str[0] + SF_str[0] + '/PCA_PC' + filename_str + add_file_str + add_file_str2 + '.svg', bbox_inches="tight")
        fig.savefig(D_str + F_str[0] + SF_str[0] + '/PCA_PC' + filename_str + add_file_str + add_file_str2 + '.pdf', bbox_inches="tight")
    plt.show()

In [1]:
def plot_PCA_2D_projection_allMutations(D_str,F_str,SF_str,DF,Y,Y_H_1,Y_H_2,var_exp,opt_mutation,bool_plotIDs,IDX_PCs,add_file_str,legend_loc,opt_sample_ID,W_H_1,W_H_2,cl,opt_save):#,n_cols,n_rows):
    if opt_mutation == 'bulk':
        mut_ending = '_B'        
    else:
        mut_ending = '_S'
    mutation_str = DF.columns[DF.columns.str.endswith(mut_ending)]
    if opt_mutation == 'combined':
        m_str = [mut_str[:-2] for mut_str in mutation_str]
        mutation_str = m_str
    
    if np.size(IDX_PCs)>2:
        n_rows = len(mutation_str)
        n_cols = np.shape(IDX_PCs)[0]
        fig, axis = plt.subplots(n_rows, n_cols, figsize=(np.ceil(n_rows/2), np.ceil(n_cols/4)))
        fig.patch.set_facecolor('white')
        fig.subplots_adjust(hspace = 0.2, wspace = 0.2) 
        for m_id in range(0,len(mutation_str)):
            comparison_str, groups, groups_colors, y, sampleIDs = get_mutation_labels(DF,mutation_str,m_id,opt_sample_ID)
            c,d = np.shape(Y)
            for cid in range(0,n_cols):
                #draw ellipse for healthy population:
                if len(Y_H_1)>1:
                    confidence_ellipse(x=Y_H_1[:,IDX_PCs[cid][0]].flatten(), y=Y_H_1[:,IDX_PCs[cid][1]].flatten(), ax=axis[m_id,cid], f_color='#57B8F9', alpha_val=0.15, W=W_H_1, confidence_levels = cl)    
                confidence_ellipse(x=Y_H_2[:,IDX_PCs[cid][0]].flatten(), y=Y_H_2[:,IDX_PCs[cid][1]].flatten(), ax=axis[m_id,cid], f_color='#194795', alpha_val=0.15, W=W_H_2, confidence_levels = cl)
                c,d = np.shape(Y)
                #for id in range(0,np.shape(y)):
                bool_first = np.ones((len(groups),1), dtype=bool)
                for name, col in groups_colors:
                    for i in range(0,c):
                        if y[i]==name:
                            g_id = groups.index(name)
                            axis[m_id,cid].scatter(Y[i,IDX_PCs[cid][0]],Y[i,IDX_PCs[cid][1]],color = col, s=90, edgecolors='#010101', label=name if bool_first[g_id] else "")  
                            if (bool_plotIDs and m_id==0):
                                axis[m_id,cid].text(Y[i,IDX_PCs[cid][0]],Y[i,IDX_PCs[cid][1]],sampleIDs[i],fontsize=10)
                            #axes[r_id,c_id] = plt.scatter(Y[i,0],Y[i,1],color = col, label=name if bool_first[g_id] else "")  
                            bool_first[g_id] = False
                #plt.setp(axis.get_xticklabels(), visible=False)
                #plt.setp(axis.get_yticklabels(), visible=False)
                axis[m_id,cid].set_xlabel('PC'+str(IDX_PCs[cid][0]+1)+' ('+str(round(var_exp[IDX_PCs[cid][0]],2))+'%)')
                axis[m_id,cid].set_ylabel('PC'+str(IDX_PCs[cid][1]+1)+' ('+str(round(var_exp[IDX_PCs[cid][1]],2))+'%)')
                if cid==0:
                    lg=axis[m_id,cid].legend(title=comparison_str, loc=legend_loc, scatterpoints=1, fontsize=15)
                    lg.get_title().set_fontsize(15)
                filename_str = '_all'
    else:
        n_cols = int(np.ceil(np.sqrt(len(mutation_str))))
        n_rows = int(np.ceil(len(mutation_str)/n_cols))
        f, axes = plt.subplots(n_rows, n_cols, figsize=(45, 35))
        f.patch.set_facecolor('white')
        f.subplots_adjust(hspace = .45, wspace = 0.2) 
        r_id=0
        c_id=0
        for m_id in range(0,len(mutation_str)):
            comparison_str, groups, groups_colors, y, sampleIDs = get_mutation_labels(DF,mutation_str,m_id,opt_sample_ID)
            c,d = np.shape(Y)
            #for id in range(0,np.shape(y)):
                    
            #draw ellipse for healthy population:
            if len(Y_H_1)>1:
                confidence_ellipse(x=Y_H_1[:,IDX_PCs[0]].flatten(), y=Y_H_1[:,IDX_PCs[1]].flatten(), ax=axes[r_id,c_id], f_color='#57B8F9', alpha_val=0.15, W=W_H_1, confidence_levels = cl)    
            confidence_ellipse(x=Y_H_2[:,IDX_PCs[0]].flatten(), y=Y_H_2[:,IDX_PCs[1]].flatten(), ax=axes[r_id,c_id], f_color='#194795', alpha_val=0.15, W=W_H_2, confidence_levels = cl)
            
            bool_first = np.ones((len(groups),1), dtype=bool)
            for name, col in groups_colors:
                for i in range(0,c):
                    if y[i]==name:
                        g_id = groups.index(name)
                        axes[r_id,c_id].scatter(Y[i,IDX_PCs[0]],Y[i,IDX_PCs[1]],color = col, s=80, edgecolors='#010101', label=name if bool_first[g_id] else "")  
                        bool_first[g_id] = False     
                        if bool_plotIDs:
                            plt.text(Y[i,IDX_PCs[0]],Y[i,IDX_PCs[1]],sampleIDs[i])
            axes[r_id,c_id].set_title('PCA: '+comparison_str)
            axes[r_id,c_id].set_xlabel('PC'+str(IDX_PCs[0]+1)+' ('+str(round(var_exp[IDX_PCs[0]],2))+'%)')
            axes[r_id,c_id].set_ylabel('PC'+str(IDX_PCs[1]+1)+' ('+str(round(var_exp[IDX_PCs[1]],2))+'%)')
            axes[r_id,c_id].legend(loc=legend_loc, scatterpoints=1)
            #axes[r_id,c_id].legend(bbox_to_anchor=(s_val, 1),numpoints=1)
            axes[r_id,c_id].grid(False)
            c_id=c_id+1
            if c_id==n_cols:
                r_id=r_id+1
                c_id=0
        while (r_id*c_id <= (n_cols-1)*(n_rows-1)):
            f.delaxes(axes[r_id,c_id])
            c_id=c_id+1
        filename_str = str(IDX_PCs[0]+1) +'_vs_PC'+ str(IDX_PCs[1]+1)
    if bool_plotIDs:
        add_file_str2 = '_withIDs'
    else:
        add_file_str2 = ''
    if opt_save:
        fig = plt.gcf() 
        fig.savefig(D_str + F_str[0] + SF_str[0] + '/PCA_mutations'+filename_str+add_file_str+add_file_str2+opt_features+'.svg', bbox_inches="tight")
        fig.savefig(D_str + F_str[0] + SF_str[0] + '/PCA_mutations'+filename_str+add_file_str+add_file_str2+opt_features+'.pdf', bbox_inches="tight")

    plt.show()

In [22]:
def get_mutation_labels(DF,mutation_str,m_id,opt_sample_ID):
    
    if opt_sample_ID=='short':
        I_col = 'Sample_ID'
    elif opt_sample_ID=='long':
        I_col = 'Sample_ID_long'
        
    a,b = np.shape(DF)
    comparison_str = mutation_str[m_id] + ' mutation'
    #labels:    
    groups=('NaN',
            mutation_str[m_id],
            'no '+ mutation_str[m_id])
    groups_colors=[('NaN','#606060'),
            (mutation_str[m_id],'#CC0000'),
            ('no '+ mutation_str[m_id],'#0080FF')]
    y=[]
    s=[]
    for i in range(0,a):
        s.append(DF[I_col][i])
        if isinstance(DF[mutation_str[m_id]][i],str): #false for nans
            if DF[mutation_str[m_id]][i]=='no': 
                y.append(groups[2]) 
            else: 
                y.append(groups[1])  
        else:
            #if individual healthy --> count as no mutation
            if np.isnan(DF[mutation_str[m_id]][i]) and DF['group'][i]=='healthy':
                y.append(groups[2]) 
            else: 
                y.append(groups[0])
    return comparison_str, groups, groups_colors, y, s

In [1]:
def plot_contribution2components(D_str,F_str,SF_str,components,n_C,n_P,ratenames,add_file_str,method_str,opt_features,opt_save):
    
    n_rows = int(np.ceil(np.sqrt(n_C)))
    n_cols = int(np.ceil(n_C/n_rows))
    if n_cols==1:
        f, axes = plt.subplots(n_rows, n_cols, figsize=(10, 30))
    else:
        f, axes = plt.subplots(n_rows, n_cols, figsize=(30, 15))
    f.subplots_adjust(hspace = .45, wspace = 0.45)   
    i_r=0
    i_c=0
    bars = tuple(ratenames)
    y_pos = np.arange(n_P)
    for i in range(0,n_rows*n_cols):
        if i<n_C:
            height = abs(components[i])
            indices = np.argsort(height)
            indices_sel = indices[len(height)-n_P:len(height)]
            bars_sorted = list()
            for indi in indices_sel:
                bars_sorted.append(bars[indi])
            if n_cols==1:
                ax=axes[i_r]
            else:
                ax=axes[i_r,i_c]

            # Create horizontal bars
            #print([y_pos,height])
            ax.barh(y_pos, height[indices_sel], color='#919191')
            # Create names on the y-axis
            ax.set_yticks(y_pos)
            ax.set_yticklabels(bars_sorted)
            ax.set_title('Absolute contribution to component '+str(i+1)) 
            ax.set_ylabel('rates')

            if i_c==n_cols-1:
                i_r=i_r+1
                i_c=0
            else:
                i_c=i_c+1
        else:
            if n_cols==1:
                f.delaxes(axes[i_r])
            else:
                f.delaxes(axes[i_r,i_c])
                i_c=i_c+1
    if opt_save:        
        fig = plt.gcf()
        fig.savefig(D_str + F_str[0] + SF_str[0] + '/'+method_str+'_contribution2components_'+add_file_str+'.pdf', bbox_inches="tight")
        fig.savefig(D_str + F_str[0] + SF_str[0] + '/'+method_str+'_contribution2components_'+add_file_str+'.svg', bbox_inches="tight")
    # Show graphic
    plt.show()

In [ ]:
def plot_contribution2components_stacked(D_str,F_str,SF_str,components,n_C,n_P,ratenames,add_file_str,method_str,opt_features,my_cmap,opt_save):    
    sum_absolute = np.sum(abs(components[0:n_C-1]),axis=0)
    inds = sum_absolute.argsort()
    ratenames_sorted = ratenames[inds]#[x for y, x in zip(sum_absolute, ratenames).sort()]
    component_sorted = abs(components[0][inds])#[abs(x) for y, x in zip(sum_absolute, components[0]).sort()]
    plt.barh([i*2 for i in range(len(ratenames_sorted))], component_sorted, height=1, color=my_cmap[0], label="PC1")
    sum_stacked = component_sorted
    for i in range(1,n_C-1):
        component_sorted = abs(components[i][inds])#[abs(x) for y, x in zip(sum_absolute, components[i]).sort()]
        #plt.barh(ratenames_sorted, component_sorted, left=sum_stacked)
        plt.barh([i*2 for i in range(len(ratenames_sorted))], component_sorted, height=1, left=sum_stacked, color=my_cmap[i], label="PC"+str(i+1))
        sum_stacked = [a + b for a, b in zip(sum_stacked,component_sorted)] 
    plt.yticks([i*2 for i in range(len(ratenames_sorted))], ratenames_sorted, fontsize=13)
    plt.ylabel('Rates')
    plt.xlabel('Absolute contribution')
    plt.ylim([-1,2*len(ratenames_sorted)-1])
    # add legend
    plt.legend(bbox_to_anchor=(-1,1),numpoints=1)
    if opt_save:        
        fig = plt.gcf()
        fig.set_size_inches(8, 10, forward=True)
        fig.savefig(D_str + F_str[0] + SF_str[0] + '/'+method_str+'_contribution2components_stacked_'+add_file_str+'.pdf', bbox_inches="tight")
        fig.savefig(D_str + F_str[0] + SF_str[0] + '/'+method_str+'_contribution2components_stacked_'+add_file_str+'.svg', bbox_inches="tight")
    # Show graphic
    plt.show()

In [25]:
def plot_explained_variance(D_str,F_str,SF_str,var_exp,add_file_str,opt_save):
    cum_var_exp = np.cumsum(var_exp)
    bar_str = list()
    for i in range(0,len(var_exp)):
        bar_str.append('PC'+str(i+1))
    bars = tuple(bar_str)
    y_pos = np.arange(len(bars))
    # Create bars
    plt.bar(y_pos, var_exp, label = 'individual', color='#919191')
    # Create names on the x-axis
    plt.xticks(y_pos, bars, rotation='vertical')
    plt.xlabel('Principal components')
    plt.ylabel('Explained variance')
    plt.plot(y_pos,cum_var_exp,'o-',Color='black', label = 'cumulative')
    #plt.title('Explained variance')
    plt.legend(loc="center right")
    if opt_save:
        fig = plt.gcf()
        fig.savefig(D_str + F_str[0] + SF_str[0] + '/PCA_explained_variance'+add_file_str+'.pdf', bbox_inches="tight")
        fig.savefig(D_str + F_str[0] + SF_str[0] + '/PCA_explained_variance'+add_file_str+'.svg', bbox_inches="tight")
     # Show graphic
    plt.show()

In [6]:
def plot_metrics(path_matlab_result,DF,par_T,I_str,I_CT_list,cols_CT,bool_interrupt_axis,MS,group,opt_sample_ID,opt_save):
    if group=='healthy':
        DF_sel=DF[DF[I_col].str.startswith('H')].copy()
    if group=='healthy without CHIP':
        mutation_str = DF.columns[DF.columns.str.endswith('VAF')]
        #print(mutation_str)
        DF_sel=DF[(DF[I_col].str.startswith('H')) & (np.any(DF[mutation_str].values.astype(np.float)>0,axis=1)==False)].copy()
    LW=2
    offset_t = 0.25
    if I_str == '_cellular_exit_time':
        scaling=[99/100,98/100, 96/100, 95/100]
    else:
        scaling=[47/50,44/50, 41/50, 38/50]
    rn_vec_sel = np.random.uniform(0.75,1.25,len(DF_sel['Sample_ID']))

    if par_T=='lin':
        if bool_interrupt_axis:
            f, axes = plt.subplots(4, len(I_CT_list), sharex=True, figsize=(4*len(I_CT_list),5))
            gs = f.add_gridspec(4,len(I_CT_list))
            f.subplots_adjust(wspace = .55)
            for i_ct,I_CT in enumerate(I_CT_list):
                axes[0,i_ct].axis('off')
                axes[1,i_ct].axis('off')
                axes[2,i_ct].axis('off')
                axes[3,i_ct].axis('off')
                
                if I_CT=='HSC' and I_str == '_accumulation_time':
                    ax_u = f.add_subplot(gs[:3,i_ct])
                    ax_l = f.add_subplot(gs[3,i_ct])
                elif I_CT=='MEP':
                    ax_l = f.add_subplot(gs[2:,i_ct])
                    ax_u = f.add_subplot(gs[:2,i_ct])
                else:
                    ax_l = f.add_subplot(gs[1:,i_ct])
                    ax_u = f.add_subplot(gs[0,i_ct])
                ax_u.get_xaxis().set_visible(True)
                ax_l.get_yaxis().set_visible(True)
                for ax in [ax_l,ax_u]:
                        ax.plot(rn_vec_sel, DF_sel[I_CT +I_str], ls='',marker='o', ms=MS, color=cols_CT[i_ct], label = group)
                        ax.plot([0.6,1.4],np.repeat(np.median(DF_sel[I_CT +I_str]),2),linestyle = '-', color = 'k', linewidth = LW, label = 'median')
                if I_str == '_cellular_exit_time':
                    if I_CT=='HSC':
                        ax_l.set_ylim(0,8)  # outliers only
                        ax_u.set_ylim(23, 25)
                    elif I_CT=='MPP':
                        ax_l.set_ylim(0,8)  # outliers only
                        ax_u.set_ylim(13, 15)
                    elif I_CT=='CMP':
                        ax_l.set_ylim(0,90)  # outliers only
                        ax_u.set_ylim(150, 180)
                    elif I_CT=='GMP':
                        ax_l.set_ylim(0,8)  # outliers only
                        ax_u.set_ylim(249, 251)
                    elif I_CT=='MEP':
                        ax_l.set_ylim(0,100)  # outliers only
                        ax_u.set_ylim(160, 260)
                elif I_str == '_accumulation_time':
                    if I_CT=='HSC':
                        ax_l.set_ylim(-4500,-3500)  # outliers only
                        ax_u.set_ylim(-2000, 1000)
                    elif I_CT=='MPP':
                        ax_l.set_ylim(0,8)  # outliers only
                        ax_u.set_ylim(13, 15)
                    elif I_CT=='CMP':
                        ax_l.set_ylim(0,16)  # outliers only
                        ax_u.set_ylim(164, 168)
                    elif I_CT=='GMP':
                        ax_l.set_ylim(0,8)  # outliers only
                        ax_u.set_ylim(249, 251)
                    elif I_CT=='MEP':
                        ax_l.set_ylim(0,100)  # outliers only
                        ax_u.set_ylim(160, 260)
                        
                # hide the spines between ax and ax2
                d = .015  # how big to make the diagonal lines in axes coordinates
                # arguments to pass to plot, just so we don't keep repeating them
                kwargs = dict(transform=ax.transAxes, color='k', clip_on=False)
                ax_u.plot((-d, +d), (-d, +d), **kwargs)        # top-left diagonal
                ax_u.plot((1 - d, 1 + d), (-d, +d), **kwargs)  # top-right diagonal

                kwargs.update(transform=ax_l.transAxes)  # switch to the bottom axes
                ax_l.plot((-d, +d), (1 - d, 1 + d), **kwargs)  # bottom-left diagonal
                ax_l.plot((1 - d, 1 + d), (1 - d, 1 + d), **kwargs)  # bottom-right diagonal

                ax_u.spines['bottom'].set_visible(False)
                ax_l.spines['top'].set_visible(False)
                ax_u.xaxis.tick_top()
                ax_l.xaxis.tick_bottom()
                
                if opt_comparison=='H_young_vs_H_aged_without_CHIP':
                    ax_u.set_xticks([1])
                    ax_u.set_xticklabels([''],rotation=90)
                    ax_l.set_xticks([1])
                    ax_l.set_xticklabels([group],rotation=90)
      
                ax_u.set_title(I_CT)
                if i_ct==0:
                    if I_str=='_net_proliferation':
                        ax_l.set_ylabel('net proliferation')
                    elif I_str=='_cellular_exit_time':
                        ax_l.set_ylabel('cellular exit time')
                    elif I_str=='_residence_time':
                        ax_l.set_ylabel('residence time')
                    else: 
                        ax_l.set_ylabel('accumulation time')
                elif i_ct==len(I_CT_list):
                    ax_l.legend(loc='lower center') 
        else:
            f, axes = plt.subplots(1, len(I_CT_list), figsize=(3*len(I_CT_list),5),sharey=True)
            if metric_str == '_cellular_exit_time':
                f.subplots_adjust(wspace = .3)
            elif metric_str == '_accumulation_time':
                f.subplots_adjust(wspace = .5)
            else:
                f.subplots_adjust(wspace = .5)
                #f.subplots_adjust(wspace = .1)
            for i,I_CT in enumerate(I_CT_list):
                axes[i].plot(rn_vec_sel, DF_sel[I_CT +I_str], ls='',marker='o', ms=MS, color=cols_CT[i], label = group)
                axes[i].plot([0.6,1.4],np.repeat(np.median(DF_sel[I_CT +I_str]),2),linestyle = '-', color = 'k', linewidth = LW)
                axes[i].set_xticks([1])
                axes[i].set_xticklabels([group],rotation=90)
                axes[i].set_title(I_CT)
                xmin,xmax = axes[i].get_xlim()
                axes[i].plot([xmin,xmax],[0, 0],linestyle = '--',color = [0.4,0.4,0.4])

            #axes[i].legend(loc='lower center')
            if I_str=='_net_proliferation':
                axes[0].set_ylabel('net proliferation')
            elif I_str=='_cellular_exit_time':
                axes[0].set_ylabel('cellular exit time')
            elif I_str=='_residence_time':
                axes[0].set_ylabel('residence time')
            else:
                axes[0].set_ylabel('accumulation time')
    if opt_save:        
        plt.savefig(path_matlab_result+ "/Rates_"+I_str+'_'+group+".svg", bbox_inches="tight")
        plt.savefig(path_matlab_result+ "/Rates_"+I_str+'_'+group+".pdf", bbox_inches="tight")

In [ ]:
def plot_metricComparison(path_matlab_result,DF,par_T,I_str,I_CT_list,bool_interrupt_axis,MS,opt_comparison,opt_sample_ID,opt_save):
    
    if opt_comparison=='H_young_vs_H_aged_without_CHIP' or opt_comparison=='H_young_vs_H_aged':
        _, DF_H_y, DF_H_a,_,_ = get_dataframe_selection(DF,opt_comparison,opt_sample_ID)
        EPs = [1,2] 
        center = 1.5
    elif opt_comparison=='H_age_matched_vs_CHIP_vs_MDS':
        _, DF_H_a, DF_CHIP, DF_MDS, _, _, _S = get_dataframe_selection(DF,opt_comparison,opt_sample_ID)
        EPs = [1,3] 
        center = 2
        a_cor = 3
    LW=2
    alpha=np.array([0.05, 0.01, 0.001])
    offset_t = 0.25
    if I_str == '_cellular_exit_time':
        scaling=[99/100,98/100, 96/100, 95/100]
    else:
        scaling=[47/50,44/50, 41/50, 38/50]
    if opt_comparison=='H_young_vs_H_aged_without_CHIP':
        rn_vec_Hy = np.random.uniform(0.75,1.25,len(DF_H_y['Sample_ID']))
        rn_vec_Ha = np.random.uniform(1.75,2.25,len(DF_H_a['Sample_ID']))
        print(len(DF_H_y[DF_H_y['group']=='healthy']['Sample_ID']))
        print(len(DF_H_a[DF_H_a['group']=='healthy']['Sample_ID']))
    elif opt_comparison=='H_age_matched_vs_CHIP_vs_MDS':
        rn_vec_Ha = np.random.uniform(0.75,1.25,len(DF_H_a['Sample_ID']))
        rn_vec_CHIP = np.random.uniform(1.75,2.25,len(DF_CHIP['Sample_ID']))
        rn_vec_MDS = np.random.uniform(2.75,3.25,len(DF_MDS['Sample_ID']))
    if par_T=='lin':
        if bool_interrupt_axis:
            f, axes = plt.subplots(4, len(I_CT_list), sharex=True, figsize=(4*len(I_CT_list),5))
            gs = f.add_gridspec(4,len(I_CT_list))
            f.subplots_adjust(wspace = .55)
            for i_ct,I_CT in enumerate(I_CT_list):
                axes[0,i_ct].axis('off')
                axes[1,i_ct].axis('off')
                axes[2,i_ct].axis('off')
                axes[3,i_ct].axis('off')
                
                if I_CT=='HSC' and I_str == '_accumulation_time':
                    ax_u = f.add_subplot(gs[:3,i_ct])
                    ax_l = f.add_subplot(gs[3,i_ct])
                elif I_CT=='MEP':
                    ax_l = f.add_subplot(gs[2:,i_ct])
                    ax_u = f.add_subplot(gs[:2,i_ct])
                else:
                    ax_l = f.add_subplot(gs[1:,i_ct])
                    ax_u = f.add_subplot(gs[0,i_ct])
                ax_u.get_xaxis().set_visible(True)
                ax_l.get_yaxis().set_visible(True)
                for ax in [ax_l,ax_u]:
                    if opt_comparison=='H_young_vs_H_aged_without_CHIP':
                        ax.plot(rn_vec_Hy, DF_H_y[I_CT +I_str], ls='',marker='o', ms=MS, color=col_H_y, label = 'healthy young')
                        ax.plot([0.6,1.4],np.repeat(np.median(DF_H_y[I_CT +I_str]),2),linestyle = '-', color = 'k', linewidth = LW, label = 'median')
                        ax.plot(rn_vec_Ha, DF_H_a[I_CT +I_str], ls='',marker='o', ms=MS, color=col_H_a, label = 'healthy aged')
                        ax.plot([1.6,2.4],np.repeat(np.median(DF_H_a[I_CT+I_str]),2),linestyle = '-', color = 'k', linewidth = LW)
                    elif opt_comparison=='healthy_age_matched_vs_CHIP_vs_MDS':
                        ax.plot(rn_vec_Ha, DF_H_a[I_CT +I_str], ls='',marker='o', ms=MS, color=col_H_a, label = 'healthy age_matched')
                        ax.plot([0.6,1.4],np.repeat(np.median(DF_H_y[I_CT +I_str]),2),linestyle = '-', color = 'k', linewidth = LW, label = 'median')
                        ax.plot(rn_vec_CHIP, DF_CHIP[I_CT +I_str], ls='',marker='o', ms=MS, color=col_CHIP, label = 'healthy aged')
                        ax.plot([1.6,2.4],np.repeat(np.median(DF_H_a[I_CT+I_str]),2),linestyle = '-', color = 'k', linewidth = LW)
                        ax.plot(rn_vec_MDS, DF_MDS[I_CT +I_str], ls='',marker='o', ms=MS, color=col_MDS, label = 'MDS')
                        ax.plot([2.6,3.4],np.repeat(np.median(DF_MDS[I_CT +I_str]),2),linestyle = '-', color = 'k', linewidth = LW)
                if I_str == '_cellular_exit_time':
                    if I_CT=='HSC':
                        ax_l.set_ylim(0,8)  # outliers only
                        ax_u.set_ylim(23, 25)
                    elif I_CT=='MPP':
                        ax_l.set_ylim(0,8)  # outliers only
                        ax_u.set_ylim(13, 15)
                    elif I_CT=='CMP':
                        ax_l.set_ylim(0,90)  # outliers only
                        ax_u.set_ylim(150, 180)
                    elif I_CT=='GMP':
                        ax_l.set_ylim(0,8)  # outliers only
                        ax_u.set_ylim(249, 251)
                    elif I_CT=='MEP':
                        ax_l.set_ylim(0,100)  # outliers only
                        ax_u.set_ylim(160, 260)
                elif I_str == '_accumulation_time':
                    if I_CT=='HSC':
                        ax_l.set_ylim(-4500,-3500)  # outliers only
                        ax_u.set_ylim(-2000, 1000)
                    elif I_CT=='MPP':
                        ax_l.set_ylim(0,8)  # outliers only
                        ax_u.set_ylim(13, 15)
                    elif I_CT=='CMP':
                        ax_l.set_ylim(0,16)  # outliers only
                        ax_u.set_ylim(164, 168)
                    elif I_CT=='GMP':
                        ax_l.set_ylim(0,8)  # outliers only
                        ax_u.set_ylim(249, 251)
                    elif I_CT=='MEP':
                        ax_l.set_ylim(0,100)  # outliers only
                        ax_u.set_ylim(160, 260)
                        
                # hide the spines between ax and ax2
                d = .015  # how big to make the diagonal lines in axes coordinates
                # arguments to pass to plot, just so we don't keep repeating them
                kwargs = dict(transform=ax.transAxes, color='k', clip_on=False)
                ax_u.plot((-d, +d), (-d, +d), **kwargs)        # top-left diagonal
                ax_u.plot((1 - d, 1 + d), (-d, +d), **kwargs)  # top-right diagonal

                kwargs.update(transform=ax_l.transAxes)  # switch to the bottom axes
                ax_l.plot((-d, +d), (1 - d, 1 + d), **kwargs)  # bottom-left diagonal
                ax_l.plot((1 - d, 1 + d), (1 - d, 1 + d), **kwargs)  # bottom-right diagonal

                ax_u.spines['bottom'].set_visible(False)
                ax_l.spines['top'].set_visible(False)
                ax_u.xaxis.tick_top()
                ax_l.xaxis.tick_bottom()
                
                if opt_comparison=='H_young_vs_H_aged_without_CHIP':
                    ax_u.set_xticks([1,2])
                    ax_u.set_xticklabels(['',''],rotation=90)
                    ax_l.set_xticks([1,2])
                    ax_l.set_xticklabels(['healthy young','healthy aged'],rotation=90)
                elif opt_comparison=='H_age_matched_vs_CHIP_vs_MDS':    
                    ax_u.set_xticks([1,2,3])
                    ax_u.set_xticklabels(['','',''],rotation=90)
                    ax_l.set_xticks([1,2,3])
                    ax_l.set_xticklabels(['healthy age-matched','CHIP','MDS'],rotation=90)
                #ax_u.tick_params(axis='x',labeltop='off')  # don't put tick labels at the top
                
                ax_u.set_title(I_CT)
                if i_ct==0:
                    if I_str=='_net_proliferation':
                        ax_l.set_ylabel('net proliferation')
                    elif I_str=='_cellular_exit_time':
                        ax_l.set_ylabel('cellular exit time')
                    elif I_str=='_residence_time':
                        ax_l.set_ylabel('residence time')
                    else: 
                        ax_l.set_ylabel('accumulation time')
                elif i_ct==len(I_CT_list):
                    ax_l.legend(loc='lower center')
                    
                y_val_min,y_val_max = ax_u.get_ylim()
                y_val1 = y_val_max*scaling[0]
                y_val1_line = y_val_max*scaling[1]
                y_val2 = y_val_max*scaling[2]
                y_val2_line = y_val_max*scaling[3]
                
                if opt_comparison=='H_young_vs_H_aged_without_CHIP':
                    TR = stats.kruskal(DF_H_y[I_CT +I_str].dropna(axis=0), DF_H_a[I_CT +I_str].dropna(axis=0))
                elif opt_comparison=='H_age_matched_vs_CHIP_vs_MDS':
                    TR = stats.kruskal(DF_H_a[I_CT +I_str].dropna(axis=0), DF_CHIP[I_CT +I_str].dropna(axis=0), DF_MDS[I_CT +I_str].dropna(axis=0))
                if TR[1]<alpha[0]:
                    print('Kruskal Wallis test:')
                    print('p-value overall differences between groups: '+str(TR[1]))
                    ax_u.plot(EPs,[y_val1_line, y_val1_line],linestyle = '-',color='black',linewidth = np.max([1,LW-1]))
                    if TR[1]<alpha[0] and TR[1]>alpha[1]:
                        ax_u.plot([center],[y_val1],linestyle = '',marker = '*',markersize = MS+2, color='black')
                    elif TR[1]<alpha[1] and TR[1]>alpha[2]:
                        ax_u.plot([center-offset_t/4,center+offset_t/4],[y_val1, y_val1],linestyle = '',marker = '*',markersize = MS+2, color='black')
                    else:
                        ax_u.plot([center-offset_t/2,center,center+offset_t/2],[y_val1, y_val1, y_val1],linestyle = '',marker = '*',markersize = MS+2, color='black')
                    #post-hoc tests:
                    if opt_comparison=='H_age_matched_vs_CHIP_vs_MDS':
                        if np.mean(DF_H_a[I_CT +I_str].dropna(axis=0))<np.mean(DF_CHIP[I_CT +I_str].dropna(axis=0)):
                            opt_hyp = 'less'
                        else:
                            opt_hyp = 'greater'
                        TR_MWU_HC = stats.mannwhitneyu(DF_H_a[I_CT +I_str].dropna(axis=0), DF_H_CHIP[I_CT +I_str].dropna(axis=0), alternative=opt_hyp) #0 and 1
                        if np.mean(DF_CHIP[I_CT +I_str].dropna(axis=0))<np.mean(DF_MDS[I_CT +I_str].dropna(axis=0)):
                            opt_hyp = 'less'
                        else:
                            opt_hyp = 'greater'
                        TR_MWU_CM = stats.mannwhitneyu(DF_CHIP[I_CT +I_str].dropna(axis=0), DF_MDS[I_CT +I_str].dropna(axis=0), alternative=opt_hyp)
                        if np.mean(DF_H_a[I_CT +I_str].dropna(axis=0))<np.mean(DF_MDS[I_CT +I_str].dropna(axis=0)):
                            opt_hyp = 'less'
                        else:
                            opt_hyp = 'greater'
                        TR_MWU_HM = stats.mannwhitneyu(DF_H_a[I_CT +I_str].dropna(axis=0), DF_MDS[I_CT +I_str].dropna(axis=0), alternative=opt_hyp)
                        if TR_MWU_HC[1]<alpha[0]/a_cor or TR_MWU_CM[1]<alpha[0]/a_cor or TR_MWU_HM[1]<alpha[0]/a_cor:
                            print('post-hoc Mann Whitney U-test: ')
                            print('p-value healthy age-matched vs. CHIP: '+str(TR_MWU_HC[1]))
                            print('p-value CHIP vs. MDS: '+str(TR_MWU_CM[1]))
                            print('p-value healthy age-matched vs. MDS: '+str(TR_MWU_HM[1]))
                            if TR_MWU_CM[1]<alpha[0]/a_cor:
                                EP2s = [2,3]
                                center2 = 2.5
                            elif TR_MWU_HC[1]<alpha[0]/a_cor:
                                EP2s = [1,2]
                                center2 = 1.5
                            else:
                                EP2s = [1,3]
                                center2 = 2
                            ax_u.plot(EP2s,[y_val2_line, y_val2_line],linestyle = '-',color='black',linewidth = np.max([1,LW-1]))
                            if TR_MWU_HC[1]<alpha[0]/a_cor and TR_MWU_HC[1]>alpha[1]/a_cor:
                                ax_u.plot([center2],[y_val2],linestyle = '',marker = '*',markersize = MS+2, color='black')
                            elif TR_MWU_HC[1]<alpha[1]/a_cor and TR_MWU_HC[1]>alpha[2]/a_cor:
                                ax_u.plot([center2-offset_t/4,center2+offset_t/4],[y_val2, y_val2],linestyle = '',marker = '*',markersize = MS+2, color='black')
                            else:
                                ax_u.plot([center2-offset_t/2,center2,center2+offset_t/2],[y_val2, y_val2, y_val2],linestyle = '',marker = '*',markersize = MS+2, color='black')

        else:
            f, axes = plt.subplots(1, len(I_CT_list), figsize=(3*len(I_CT_list),5))
            #f, axes = plt.subplots(1, len(I_CT_list), sharex=True,figsize=(3*len(I_CT_list),5))
            if metric_str == '_cellular_exit_time':
                f.subplots_adjust(wspace = .3)
            elif metric_str == '_accumulation_time':
                f.subplots_adjust(wspace = .5)
            else:
                f.subplots_adjust(wspace = .5)
                #f.subplots_adjust(wspace = .1)
            for i,I_CT in enumerate(I_CT_list):
                if opt_comparison=='H_young_vs_H_aged_without_CHIP':
                    axes[i].plot(rn_vec_Hy, DF_H_y[I_CT +I_str], ls='',marker='o', ms=MS, color=col_H_y, label = 'healthy young')
                    axes[i].plot([0.6,1.4],np.repeat(np.median(DF_H_y[I_CT +I_str]),2),linestyle = '-', color = 'k', linewidth = LW)
                    axes[i].plot(rn_vec_Ha, DF_H_a[I_CT +I_str], ls='',marker='o', ms=MS, color=col_H_a, label = 'healthy aged')
                    axes[i].plot([1.6,2.4],np.repeat(np.median(DF_H_a[I_CT +I_str]),2),linestyle = '-', color = 'k', linewidth = LW)
                    axes[i].set_xticks([1,2])
                    axes[i].set_xticklabels(['healthy young','healthy aged'],rotation=90)
                elif opt_comparison=='H_age_matched_vs_CHIP_vs_MDS':    
                    axes[i].plot(rn_vec_Ha, DF_H_a[I_CT +I_str], ls='',marker='o', ms=MS, color=col_H_a, label = 'healthy age-matched')
                    axes[i].plot([0.6,1.4],np.repeat(np.median(DF_H_a[I_CT +I_str]),2),linestyle = '-', color = 'k', linewidth = LW)
                    axes[i].plot(rn_vec_CHIP, DF_CHIP[I_CT +I_str], ls='',marker='o', ms=MS, color=col_CHIP, label = 'CHIP')
                    axes[i].plot([1.6,2.4],np.repeat(np.median(DF_CHIP[I_CT +I_str]),2),linestyle = '-', color = 'k', linewidth = LW)
                    axes[i].plot(rn_vec_MDS, DF_MDS[I_CT +I_str], ls='',marker='o', ms=MS, color=col_MDS, label = 'MDS')
                    axes[i].plot([2.6,3.4],np.repeat(np.median(DF_MDS[I_CT +I_str]),2),linestyle = '-', color = 'k', linewidth = LW, label = 'median')
                    axes[i].set_xticks([1,2,3])
                    axes[i].set_xticklabels(['healthy age-matched','CHIP','MDS'],rotation=90)
                axes[i].set_title(I_CT)
                #if I_str=='_cellular_exit_time' and I_CT=='MEP' or I_CT=='GMP':
                #    axes[i].set_ylim([0,300])
                #if I_str=='_net_proliferation':
                #    axes[i].set_ylim([-0.15,0.15])
                    #axes[i].set_ylim([-1.1,1.1])
                
                if opt_comparison=='H_young_vs_H_aged_without_CHIP' or opt_comparison=='H_young_vs_H_aged':
                    TR = stats.kruskal(DF_H_y[I_CT +I_str].dropna(axis=0), DF_H_a[I_CT +I_str].dropna(axis=0))
                elif opt_comparison=='H_age_matched_vs_CHIP_vs_MDS':   
                    TR = stats.kruskal(DF_H_a[I_CT +I_str].dropna(axis=0), DF_CHIP[I_CT +I_str].dropna(axis=0), DF_MDS[I_CT +I_str].dropna(axis=0))
                y_val_min,y_val_max = axes[i].get_ylim()
                y_val1 = y_val_max*scaling[0]
                y_val1_line = y_val_max*scaling[1]
                y_val2 = y_val_max*scaling[2]
                y_val2_line = y_val_max*scaling[3]
                if TR:
                    if TR[1]<alpha[0]:
                        print('Kruskal Wallis test:')
                        print('p-value overall differences between groups: '+ str(TR[1]))
                        axes[i].plot(EPs,[y_val1_line, y_val1_line],linestyle = '-',color='black',linewidth = np.max([1,LW-1]))
                        if TR[1]<alpha[0] and TR[1]>alpha[1]:
                            axes[i].plot([center],[y_val1],linestyle = '',marker = '*',markersize = MS+2, color='black')
                        elif TR[1]<alpha[1] and TR[1]>alpha[2]:
                            axes[i].plot([center-offset_t/4,center+offset_t/4],[y_val1, y_val1],linestyle = '',marker = '*',markersize = MS+2, color='black')
                        else:
                            axes[i].plot([center-offset_t/2,center,center+offset_t/2],[y_val1, y_val1, y_val1],linestyle = '',marker = '*',markersize = MS+2, color='black')
                        if opt_comparison=='H_age_matched_vs_CHIP_vs_MDS':  
                            #post-hoc tests:
                            if np.mean(DF_H_a[I_CT +I_str].dropna(axis=0))<np.mean(DF_CHIP[I_CT +I_str].dropna(axis=0)):
                                opt_hyp = 'less'
                            else:
                                opt_hyp = 'greater'
                            TR_MWU_HC = stats.mannwhitneyu(DF_H_a[I_CT +I_str].dropna(axis=0), DF_CHIP[I_CT +I_str].dropna(axis=0), alternative=opt_hyp) #0 and 1
                            if np.mean(DF_CHIP[I_CT +I_str].dropna(axis=0))<np.mean(DF_MDS[I_CT +I_str].dropna(axis=0)):
                                opt_hyp = 'less'
                            else:
                                opt_hyp = 'greater'
                            TR_MWU_CM = stats.mannwhitneyu(DF_CHIP[I_CT +I_str].dropna(axis=0), DF_MDS[I_CT +I_str].dropna(axis=0), alternative=opt_hyp)
                            if np.mean(DF_H_a[I_CT +I_str].dropna(axis=0))<np.mean(DF_MDS[I_CT +I_str].dropna(axis=0)):
                                opt_hyp = 'less'
                            else:
                                opt_hyp = 'greater'
                            TR_MWU_HM = stats.mannwhitneyu(DF_H_a[I_CT +I_str].dropna(axis=0), DF_MDS[I_CT +I_str].dropna(axis=0), alternative=opt_hyp)
                            if TR_MWU_HC[1]<alpha[0]/a_cor or TR_MWU_CM[1]<alpha[0]/a_cor or TR_MWU_HM[1]<alpha[0]/a_cor:
                                print('post-hoc Mann Whitney U-test: ')
                                print('p-value healthy age-matched vs. CHIP: '+ str(TR_MWU_HC[1]))
                                print('p-value CHIP vs. MDS: '+ str(TR_MWU_CM[1]))
                                print('p-value healthy age-matched vs. MDS: '+str(TR_MWU_HM[1]))
                                if TR_MWU_CM[1]<alpha[0]/a_cor:
                                    EP2s = [2,3]
                                    center2 = 2.5
                                elif TR_MWU_HC[1]<alpha[0]/a_cor:
                                    EP2s = [1,2]
                                    center2 = 1.5
                                else:
                                    EP2s = [1,3]
                                    center2 = 2
                                axes[i].plot(EP2s,[y_val2_line, y_val2_line],linestyle = '-',color='black',linewidth = np.max([1,LW-1]))
                                if TR_MWU_HC[1]<alpha[0]/a_cor and TR_MWU_HC[1]>alpha[1]/a_cor:
                                    axes[i].plot([center2],[y_val2],linestyle = '',marker = '*',markersize = MS+2, color='black')
                                elif TR_MWU_HC[1]<alpha[1]/a_cor and TR_MWU_HC[1]>alpha[2]/a_cor:
                                    axes[i].plot([center2-offset_t/4,center2+offset_t/4],[y_val2, y_val2],linestyle = '',marker = '*',markersize = MS+2, color='black')
                                else:
                                    axes[i].plot([center2-offset_t/2,center2,center2+offset_t/2],[y_val2, y_val2, y_val2],linestyle = '',marker = '*',markersize = MS+2, color='black')
                xmin,xmax = axes[i].get_xlim()
                axes[i].plot([xmin,xmax],[0, 0],linestyle = '--',color = [0.4,0.4,0.4])
                #if i>0 and I_str=='_net_proliferation':
                #    axes[i].set_yticklabels([])  

            #axes[i].legend(loc='lower center')
            if I_str=='_net_proliferation':
                axes[0].set_ylabel('net proliferation')
            elif I_str=='_cellular_exit_time':
                axes[0].set_ylabel('cellular exit time')
            elif I_str=='_residence_time':
                axes[0].set_ylabel('residence time')
            else:
                axes[0].set_ylabel('accumulation time')
    if opt_save:        
        plt.savefig(path_matlab_result+ "/RateComparison_"+I_str+opt_comparison+".svg", bbox_inches="tight")
        plt.savefig(path_matlab_result+ "/RateComparison_"+I_str+opt_comparison+".pdf", bbox_inches="tight")

In [ ]:
def plot_metric_vs_age(metric_str, CT_list, group, DF, opt_save,path_matlab_result):
    x=DF[DF['group']==group]['age']
    f, axes = plt.subplots(1, len(CT_list), figsize=(30, 5))
    f.patch.set_facecolor('white')
    f.subplots_adjust(hspace = .25, wspace = 0.2)                       
    for cid,ct in enumerate(CT_list):
        y=DF[DF['group']==group][ct+'_'+metric_str]
        rho,pval = stats.spearmanr(x,y)
        axes[cid].plot(x,y,'.',ms=12)
        if metric_str =='net_proliferation':
            y_val_text=0.21
        ymin,ymax = axes[cid].get_ylim()
        axes[cid].text(60,ymax+0.02*(ymax-ymin), '(rho = '+str(np.round(rho,2))+')')
        axes[cid].set_title(ct)
        axes[cid].set_xlabel('age')
    axes[0].set_ylabel(metric_str)
    if opt_save:
        f = plt.gcf()
        f.savefig(path_matlab_result + '/' + 'Metric_' + metric_str + '_vs_age'+ '.svg', bbox_inches="tight")
        f.savefig(path_matlab_result + '/' + 'Metric_' + metric_str + '_vs_age'+ '.pdf', bbox_inches="tight")
    plt.show()

In [7]:
def getWeightDataframe(W_raw,W_data):
    W = W_raw + (np.tile(W_data,(np.shape(W_raw)[1],1))).T #uncomment if number of used data points should be used as weight
    W = W/np.sum(W)
    return W;

In [ ]:
def loadPltSettings(fontSize,markerSize):
    plt.style.use('classic')
    plt.rcParams['grid.alpha'] = 0
    plt.rcParams['font.size'] = fontSize
    plt.rcParams['axes.edgecolor'] ='black'
    plt.rcParams['axes.facecolor'] ='white'
    plt.rcParams['axes.labelcolor'] = 'black'
    plt.rcParams['figure.edgecolor'] = 'white'
    plt.rcParams['figure.facecolor'] = 'white'
    plt.rcParams['figure.titlesize'] = fontSize
    plt.rcParams['font.family'] = 'Arial'
    plt.rcParams['axes.labelsize']= fontSize
    plt.rcParams['legend.fontsize'] = fontSize
    plt.rcParams['boxplot.flierprops.markersize'] = markerSize
    plt.rcParams['lines.markersize'] = markerSize
    return plt